In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_91708/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import dynamo as dyn
dyn.configuration.set_figure_params('dynamo', background='white')
print(dyn.__path__)
import pandas as pd
import numpy as np
from scipy import sparse
from scipy import stats
from scipy.integrate import solve_ivp
from dynamo.vectorfield import scVectorField
import math
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as mpatches
import random
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import seaborn as sns
import scanpy as sc
import pickle
import os
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from dynamo.vectorfield.utils import vector_transformation
from dynamo.preprocessing.cell_cycle import get_cell_phase_genes
import collections
from scipy.sparse import issparse
from collections import Counter
from matplotlib import ticker
from scipy.stats import ks_2samp
import anndata as ad
import umap

In [2]:
dyn.get_all_dependencies_version()

package,umap-learn,tqdm,statsmodels,setuptools,seaborn,scipy,scikit-learn,python-igraph,pynndescent,pre-commit,pandas,openpyxl,numpy,numdifftools,numba,networkx,matplotlib,loompy,get-version,dynamo-release,cvxopt,colorcet
version,0.5.5,4.66.1,0.14.1,69.0.3,0.13.1,1.10.1,1.3.2,0.11.3,0.5.11,3.5.0,2.0.3,3.1.2,1.24.4,0.9.41,0.58.1,3.1,3.7.4,3.0.7,3.5.4,1.1.0,1.3.2,3.0.1


In [1]:
#Define a nice colour map for gene expression
from matplotlib import colors
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

NameError: name 'plt' is not defined

In [ ]:
phases = ['G1.S', 'S', 'G2', 'G2.M', 'M.G1']
colors = {'G1.S':'firebrick', 'S':'orange', 'G2':'yellowgreen', 'G2.M':'teal', 'M.G1':'royalblue'}
doses = [0,12,25,50,100,200,400,800]

## Processing adata

In [1]:
# Processing the data - individual UMAP reference
def processAdata(adata, density = False):
    dyn.tl.dynamics(adata, model='deterministic', cores=3)
    dyn.tl.reduceDimension(adata, enforce=True)
    dyn.tl.cell_wise_confidence(adata)
    dyn.tl.cell_velocities(adata, method='pearson', enforce=True, other_kernels_dict={'transform': 'sqrt'}, correct_density = density)
    dyn.tl.cell_velocities(adata, basis = 'pca', enforce = True, method='pearson', other_kernels_dict={'transform': 'sqrt'}, correct_density = density)
    #dyn.tl.cell_velocities(adata, basis = 'DC', enforce = True, method='pearson', other_kernels_dict={'transform': 'sqrt'}, correct_density = density)
    #visualize what the velocity looks like
    dyn.vf.VectorField(adata, basis='pca', M=1000)
    dyn.vf.VectorField(adata, basis='umap', M=1000)
    dyn.vf.topography(adata, basis='umap')
    dyn.vf.topography(adata, basis='pca')
    return adata

In [ ]:
# adata = not processed
# processed_adata = processed adata (contains the UMAP and PCA coordinate reference)
def processAdata_sameRef(adata, processed_adata, dosage, genes, keep_filtered = False, density = False):

    adata_dose = adata[(dosage == adata.obs['dosage'])]
    if genes is None:
        dyn.pp.recipe_monocle(adata_dose, n_top_genes=1000, keep_filtered_genes=True)
    else:
        #adata_dose.var.index = adata_dose.var['var_names']
        dyn.pp.recipe_monocle(adata_dose, genes_to_use=genes,  n_top_genes = len(genes), keep_filtered_genes=keep_filtered)
    
    X_umap = processed_adata.obsm['X_umap']
    X_pca = processed_adata.obsm['X_pca']
    
    index = dosage == processed_adata.obs['dosage']
    adata_dose.obsm['X_pca'] = X_pca[index]
    
    dyn.tl.dynamics(adata_dose, model='deterministic', cores=3)
    
    adata_dose.obsm['X_umap'] = X_umap[index]
    
    dyn.tl.reduceDimension(adata_dose)
    
    dyn.tl.cell_velocities(adata_dose, method='pearson', enforce=True, other_kernels_dict={'transform': 'sqrt'}, correct_density = density)
    dyn.tl.cell_velocities(adata_dose, basis = 'pca', enforce = True, method='pearson', other_kernels_dict={'transform': 'sqrt'}, correct_density=density)
    dyn.vf.VectorField(adata_dose, basis='pca', M=1000)
    dyn.vf.VectorField(adata_dose, basis='umap', M=1000)
    dyn.vf.topography(adata_dose, basis='pca')
    dyn.vf.topography(adata_dose, basis='umap')
    
    return adata_dose

In [175]:
# adata = not processed
# processed_adata = processed adata (contains the UMAP and PCA coordinate reference)
def processAdata_sameRef2(adata, processed_adata, dosage, genes, dims=30, keep_filtered = False, density = False):
    print('Dosage:', dosage)
    adata_dose = adata[(dosage == adata.obs['dosage'])]
    #adata.var.index = adata.var['var_names']
    if genes is None:
        dyn.pp.recipe_monocle(adata_dose, n_top_genes=1000, keep_filtered_genes=False)
    else:
        adata_dose.var.index = adata_dose.var['var_names']
        dyn.pp.recipe_monocle(adata_dose, genes_to_use=genes,  n_top_genes = len(genes), keep_filtered_genes=keep_filtered)

    X_umap = processed_adata.obsm['X_umap']
    X_pca = processed_adata.obsm['X_pca'][:,0:dims]
    
    index = dosage == processed_adata.obs['dosage']

    dyn.tl.dynamics(adata_dose, model='deterministic', cores=3)

    adata_dose.obsm['X_pca'] = X_pca[index]
    adata_dose.obsm['X_umap'] = X_umap[index]

    adata_dose.uns['pca_fit'] = processed_adata.uns['pca_fit']
    adata_dose.uns['umap_fit'] = processed_adata.uns['umap_fit']
    
    #dyn.tl.reduceDimension(adata_dose)
    dyn.tl.neighbors(adata_dose)
    dyn.tl.cell_velocities(adata_dose, method='pearson', enforce=True, other_kernels_dict={'transform': 'sqrt'}, correct_density = density)
    dyn.tl.cell_velocities(adata_dose, basis = 'pca', enforce = True, method='pearson', other_kernels_dict={'transform': 'sqrt'}, correct_density=density)
    dyn.vf.VectorField(adata_dose, basis='pca', M=1000)
    dyn.vf.VectorField(adata_dose, basis='umap', M=1000)
    dyn.vf.topography(adata_dose, basis='pca')
    dyn.vf.topography(adata_dose, basis='umap')
    
    return adata_dose

In [181]:
def process_ind(adata, dosage, genes, density):
    adata_dose = adata[(dosage == adata.obs['dosage'])]
    if genes is None:
        dyn.pp.recipe_monocle(adata_dose, n_top_genes=1000, keep_filtered_genes=False)
    else:
        adata_dose.var.index = adata_dose.var['var_names']
        dyn.pp.recipe_monocle(adata_dose, genes_to_use=genes, keep_filtered_genes=density)

    dyn.tl.dynamics(adata_dose, model='deterministic', cores=3)
    dyn.tl.reduceDimension(adata_dose)
    dyn.tl.cell_wise_confidence(adata_dose)
    dyn.tl.cell_velocities(adata_dose, method='pearson', enforce=True, other_kernels_dict={'transform': 'sqrt'}, correct_density = density)
    dyn.tl.cell_velocities(adata_dose, basis = 'pca', enforce = True, method='pearson', other_kernels_dict={'transform': 'sqrt'}, correct_density = density)
    dyn.vf.VectorField(adata_dose, basis='pca', M=1000)
    dyn.vf.topography(adata_dose, basis='umap')
    dyn.vf.topography(adata_dose, basis='pca')
    return adata_dose

In [2]:
# Obtains the acceleration and divergence rank for all the genes 
def get_rank_genes(adata):
    valid_gene_list = adata.var['var_names'].astype(str)
    dyn.vf.jacobian(adata, regulators=valid_gene_list.index, effectors=valid_gene_list.index)
    dyn.vf.rank_acceleration_genes(adata)
    
    temp = np.column_stack((adata.uns['rank_acceleration'].astype(int), adata.var['var_names'][:,None]))
    sorted_acceleration = temp[temp[:, 0].argsort()]
    
    dyn.vf.rank_divergence_genes(adata)
    temp = np.column_stack((adata.uns['rank_div_gene_jacobian_pca'].astype(int), adata.var['var_names'][:,None]))
    sorted_divergence = temp[temp[:, 0].argsort()]
    
    return sorted_acceleration, sorted_divergence

In [2]:
def saveToTXT(toSave, fileName):
    file = open("../generated-data/Gene Lists/"+fileName, "w")
    for i in range(toSave.shape[0]):
        #file.write(toSave[i,1].astype(str)+"\n")
        file.write(toSave[i,0].astype(str)+"\n")

In [4]:
def plot_jacobian_genes(adata, jkey="jacobian", basis="pca", regulators=None, effectors=None, **kwargs):
    jkey = f"{jkey}_{basis}" if basis is not None else jkey
    J_dict = adata.uns[jkey]
    c_arr = []
    ti_arr = []
    for i, reg in enumerate(J_dict["regulators"]):
        if regulators is None or reg in regulators:
            for j, eff in enumerate(J_dict["effectors"]):
                if effectors is None or eff in effectors:
                    if i!=j:
                        c_arr.append(J_dict["jacobian_gene"][j, i, :])
                        ti_arr.append(f"{eff} wrt. {reg}")
    dyn.pl.multiplot(
        lambda c, ti: [dyn.pl.zscatter(adata, color=c, **kwargs), plt.title(ti)],
        {"c": c_arr, "ti": ti_arr},
        n_col=2,
        subplot_size=(8, 4),
    )

In [5]:
def plot_jacobian_gene_gene(adata, eff, reg, basis="pca", jkey="jacobian", both=True, regulators=None, effectors=None, **kwargs):
    jkey = f"{jkey}_{basis}" if basis is not None else jkey
    #print(jkey)
    J_dict = adata.uns[jkey]
    i = np.where(np.array(adata.uns[jkey]["regulators"]) == reg)[0][0]
    j = np.where(np.array(adata.uns[jkey]["effectors"]) == eff)[0][0]
    #c_arr = [J_dict["jacobian_gene"][j, i, :], J_dict["jacobian_gene"][i, j, :]]
    c_arr = []
    ti_arr = []
    if both == True:
        c_arr = [J_dict["jacobian_gene"][j, i, :], J_dict["jacobian_gene"][i, j, :]]
        ti_arr = [f"{eff} wrt. {reg}", f"{reg} wrt. {eff}"]
    else:
        c_arr = [J_dict["jacobian_gene"][j, i, :]]
        ti_arr = [f"{eff} wrt. {reg}"]
    dyn.pl.multiplot(
        lambda c, ti: [dyn.pl.zscatter(adata, color=c, **kwargs), plt.title(ti)],
        {"c": c_arr, "ti": ti_arr},
        n_col=2,
        subplot_size=(8, 4),
    )

In [6]:
def getNNeighbors(coord, coords, k=150):
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(coords)
    _, neighs = nbrs.kneighbors(np.atleast_2d(coord))
    return neighs

In [7]:
# Plots the topography comparing two doses 
def compare_topography(adata_1, adata_2, geneList, doses):
    for gene in geneList:
        index = np.where(adata_1.var['var_names'] == gene)[0][0]
        _min = np.min(np.concatenate((adata_1.X[:,index].toarray(),adata_2.X[:,index].toarray()), axis=0))
        _max = np.max(np.concatenate((adata_1.X[:,index].toarray(),adata_2.X[:,index].toarray()), axis=0))

        name = gene + " gene dose " + str(doses[0])
        adata_1.obs[name] = adata_1.X[:,index].toarray()
        dyn.pl.topography(adata_1, basis='umap', color = name, background='white', streamline_color='black', show_legend='on data', frontier=True, vmin=0, vmax=_max)

        index = np.where(adata_2.var['var_names'] == gene)[0][0]
        name = gene + " gene dose " + str(doses[1])
        adata_2.obs[name] = adata_2.X[:,index].toarray()
        dyn.pl.topography(adata_2, basis='umap', color = name, background='white', streamline_color='black', show_legend='on data', frontier=True, vmin=0, vmax=_max)

In [32]:
# Plot umap with a specific genes expression values
def umap_gene(adata, gene_name):
    gene_index = np.where(adata.var['var_names'] == gene_name)[0][0]
    gene_exp = adata.X[:,gene_index].toarray().flatten().tolist()
    adata.obs[gene_name+'_gene_exp'] = gene_exp
    #dyn.pl.umap(adata, color=gene_name+'gene_exp')
    return adata

In [2]:
# Plot pca with a specific genes expression values
def pca_gene(adata, gene_name):
    gene_index = np.where(adata.var['var_names'] == gene_name)[0][0]
    gene_exp = adata.X[:,gene_index].toarray().flatten().tolist()
    adata.obs[gene_name+'gene_exp'] = gene_exp
    #dyn.pl.pca(adata, color=gene_name+'gene_exp')
    return adata

In [1]:
# Processing data with same UMAP reference
# Using a specified gene list
def processAdata_genes(adata, processed_adata, genes, dosage, density):
    
    adata_dose = adata[(dosage == adata.obs['dosage'])]
    adata_dose.var.index = adata_dose.var['var_names']
    dyn.pp.recipe_monocle(adata_dose, genes_to_use = genes, keep_filtered_genes = False)
    
    X_umap = processed_adata.obsm['X_umap']
    X_pca = processed_adata.obsm['X_pca']
    
    index = dosage == processed_adata.obs['dosage']
    adata_dose.obsm['X_pca'] = X_pca[index]
    
    dyn.tl.dynamics(adata_dose, model='deterministic', cores=3)
    
    adata_dose.obsm['X_umap'] = X_umap[index]
    
    dyn.tl.reduceDimension(adata_dose)
    
    dyn.tl.cell_velocities(adata_dose, method='pearson', enforce=True, other_kernels_dict={'transform': 'sqrt'}, correct_density = density)
    dyn.tl.cell_velocities(adata_dose, basis = 'pca', enforce = True, method='pearson', other_kernels_dict={'transform': 'sqrt'}, correct_density=density)
    dyn.vf.VectorField(adata_dose, basis='pca', M=1000)
    dyn.vf.VectorField(adata_dose, basis='umap', M=1000)
    dyn.vf.topography(adata_dose, basis='pca')
    dyn.vf.topography(adata_dose, basis='umap')
    
    return adata_dose

In [2]:
# Processing data with individiual UMAP reference
# Using a specified gene list
def processAdata_genes2(adata, processed_adata, genes, dosage, density):
    
    adata_dose = adata[(dosage == adata.obs['dosage'])]
    adata_dose.var.index = adata_dose.var['var_names']
    dyn.pp.recipe_monocle(adata_dose, genes_to_use = genes, keep_filtered_genes = False)
    
    #X_umap = processed_adata.obsm['X_umap']
    #X_pca = processed_adata.obsm['X_pca']
    
    #index = dosage == processed_adata.obs['dosage']
    #adata_dose.obsm['X_pca'] = X_pca[index]
    
    dyn.tl.dynamics(adata_dose, model='deterministic', cores=3)
    
    #adata_dose.obsm['X_umap'] = X_umap[index]
    
    dyn.tl.reduceDimension(adata_dose)
    
    dyn.tl.cell_velocities(adata_dose, method='pearson', enforce=True, other_kernels_dict={'transform': 'sqrt'}, correct_density = density)
    dyn.tl.cell_velocities(adata_dose, basis = 'pca', enforce = True, method='pearson', other_kernels_dict={'transform': 'sqrt'}, correct_density=density)
    dyn.vf.VectorField(adata_dose, basis='pca', M=1000)
    dyn.vf.VectorField(adata_dose, basis='umap', M=1000)
    dyn.vf.topography(adata_dose, basis='pca')
    dyn.vf.topography(adata_dose, basis='umap')
    
    return adata_dose

## Saving and reloading adata

In [2]:
# B is assumed to be a numpy array
def saveas_R_file(B, filename):
    rpy2.robjects.numpy2ri.activate()
    nr,nc = B.shape
    Br = ro.r.matrix(B, nrow=nr, ncol=nc)
    ro.r.assign("B", Br)
    ro.r("save(B, file='"+filename+".Rdata')")

In [158]:
def save_vector_field(adata_list, processed_adata, file_name, dict_name):
    for i in range(len(adata_list)):
        dose = np.unique(adata_list[i].obs['dosage'])[0]
        print(dose)
        temp_name = file_name + str(dose) + ".h5ad"
        #temp = processed_adata[(dose == processed_adata.obs['dosage'])]
        #adata_list[i].var.index = temp.var.index

        adata_list[i].var.index = np.arange(adata_list[i].shape[1])
        adata_list[i].var = adata_list[i].var.fillna(0)
        file = open(dict_name + str(dose) + '.pkl', 'wb')
        pickle.dump(adata_list[i].uns, file)
        file.close()

        temp_adata = adata_list[i].copy()
        temp_adata.uns = {}
        temp_adata.write_h5ad(filename=temp_name)

In [ ]:
def save_processed_adata(adata, processed_adata, file_name, dict_name):
    #dose = np.unique(adata_list[i].obs['dosage'])[0]
    #print(dose)
    temp_name = file_name + ".h5ad"
    #temp = processed_adata[(dose == processed_adata.obs['dosage'])]
    #processed_adata.var.index = adata.var.index

    file = open(dict_name + '.pkl', 'wb')
    temp_dict = {}
    for key in processed_adata.uns.keys():
        print(key)
        if key != 'neighbors':
            temp_dict[key] = processed_adata.uns[key]
    print("done loading")
    #pickle.dump(processed_adata.uns, file)
    pickle.dump(temp_dict, file)
    file.close()

    temp_adata = processed_adata.copy()
    temp_adata.uns = {}
    #adata_list[i].uns = {}
    temp_adata.write_h5ad(filename=temp_name)

In [ ]:
def save_processed_adata2(adata, processed_adata, file_name, dict_name, umap_model_name):
    #dose = np.unique(adata_list[i].obs['dosage'])[0]
    #print(dose)
    temp_name = file_name + ".h5ad"
    #temp = processed_adata[(dose == processed_adata.obs['dosage'])]
    #processed_adata.var.index = adata.var.index

    #processed_adata.uns['umap_fit']['fit'].save(umap_model_name)

    file = open(dict_name + '.pkl', 'wb')
    temp_dict = {}
    for key in processed_adata.uns.keys():
        #if key!='neighbors' or key!='umap_fit':
        #   temp_dict[key] = processed_adata.uns[key]
        if 'neighbors' not in key:
            temp_dict[key] = processed_adata.uns[key]
        
    #pickle.dump(processed_adata.uns, file)
    pickle.dump(temp_dict, file)
    file.close()

    temp_adata = processed_adata.copy()
    temp_adata.uns = {}
    #adata_list[i].uns = {}
    temp_adata.write_h5ad(filename=temp_name)

In [1]:
# INPUTS:
#   length = indicates how many doses need to loaded (defualt = 8 which means all doses are going to be loaded)
def reload_data(file_name, dict_name, length=8):
    reloaded = []
    doses = [0, 12, 25, 50, 100, 200, 400, 800]
    for i in range(length):
        temp_name = file_name + str(doses[i]) + ".h5ad"
        temp = dyn.read_h5ad(temp_name)
        loaded_dict = pd.read_pickle(dict_name + str(doses[i]) + '.pkl')
        temp.uns = loaded_dict
        reloaded.append(temp.copy())
    return reloaded

In [1]:
# INPUTS:
#   length = indicates how many doses need to loaded (defualt = 8 which means all doses are going to be loaded)
def reload_data2(file_name, dict_name, length=8):
    reloaded = []
    doses = [0, 12, 25, 50, 100, 200, 400, 800]
    for i in range(length):
        temp_name = file_name + str(doses[i]) + ".h5ad"
        temp = dyn.read_h5ad(temp_name)
        #loaded_dict = pd.read_pickle(dict_name + str(doses[i]) + '.pkl')
        #temp.uns = loaded_dict
        reloaded.append(temp.copy())
    return reloaded

In [2]:
def reload_processed_data(file_name, dict_name):
    
    temp_name = file_name + ".h5ad"
    temp = dyn.read_h5ad(temp_name)
    loaded_dict = pd.read_pickle(dict_name + '.pkl')
    temp.uns = loaded_dict
    #temp.uns['umap_fit'] = load_ParametricUMAP(umap_model_name)
    return temp

In [4]:
def reload_processed_data_new(file_name, dict_name):
    
    temp_name = file_name + ".h5ad"
    temp = dyn.read_h5ad(temp_name)
    #loaded_dict = pd.read_pickle(dict_name + '.pkl')
    #temp.uns = loaded_dict
    #temp.uns['umap_fit'] = load_ParametricUMAP(umap_model_name)
    return temp

In [3]:
def reload_processed_data2(file_name, dict_name, umap_model_name):
    
    temp_name = file_name + ".h5ad"
    temp = dyn.read_h5ad(temp_name)
    loaded_dict = pd.read_pickle(dict_name + '.pkl')
    temp.uns = loaded_dict
    temp.uns['umap_fit'] = load_ParametricUMAP(umap_model_name)
    return temp

## Fixed points

In [21]:
def getFixedPoints(adata, basis):
    basis_key = "VecFld_" + basis
    if basis == 'umap':
        if adata.uns[basis_key]['Xss'].shape[0] > 0:
            all_fps = np.hstack((adata.uns[basis_key]['Xss'][:,0:2], adata.uns[basis_key]['ftype'][:,None], adata.uns[basis_key]['confidence'][:,None]))
        else:
            return [],[],[],[]
    else:
        if adata.uns[basis_key]['X'].shape[1] == 2:
            if adata.uns[basis_key]['Xss'].shape[0] > 0:
                all_fps = np.hstack((adata.uns[basis_key]['Xss'][:,0:2], adata.uns[basis_key]['ftype'][:,None], adata.uns[basis_key]['confidence'][:,None]))
            else:
                return [],[],[],[]
        else:
            if adata.uns[basis_key]['Xss'].shape[0] > 0:
                all_fps = np.hstack((adata.uns[basis_key]['Xss'][:,0:2], adata.uns[basis_key]['ftype'][:,None]))
            else:
                return [],[],[],[]
    stable = all_fps[np.where(all_fps[:,2] == -1)]
    unstable = all_fps[np.where(all_fps[:,2] == 1)]
    saddle = all_fps[np.where(all_fps[:,2] == 0)]
    return all_fps, stable, unstable, saddle

In [5]:
# Plots the fixed points for a specific dose
# adata - the current dose 
# adata_forward - the dose after the current dose
# adata_backward - the dose before the current dose
# stable_points, unstable_points, saddle_points = fixed points of the adata current dose
def plot_fp(adata, adata_forward, adata_backward, stable_points, unstable_points, saddle_points, fileName):
    dose = adata.obs['dosage'][0]
    if adata_forward is not None:
        plt.scatter(adata_forward.obsm['X_umap'][:,0], adata_forward.obsm['X_umap'][:,1], c='gainsboro', label="next dose")
    if adata_backward is not None:
        plt.scatter(adata_backward.obsm['X_umap'][:,0], adata_backward.obsm['X_umap'][:,1], c='silver', label="previous dose")
    plt.scatter(adata.obsm['X_umap'][:,0], adata.obsm['X_umap'][:,1], c='mediumpurple', label="current dose")
    #plt.scatter(stable_points[:,0], stable_points[:,1], c='r', label='stable')
    plt.scatter(unstable_points[:,0], unstable_points[:,1], c='violet', label='unstable')
    plt.scatter(saddle_points[:,0], saddle_points[:,1], c='blue', label='saddle')
    plt.scatter(stable_points[:,0], stable_points[:,1], c='r', label='stable')
    plt.legend(bbox_to_anchor=(1.35,1), loc="upper right")
    plt.xlabel("UMAP 0")
    plt.ylabel("UMAP 1")
    plt.title("TGFB dose " + dose + " fixed points")
    #plt.savefig(fileName, bbox_inches='tight', dpi=300)
    #plt.show()

In [19]:
def gene_acceleration(adata, gene_name):
    gene_index = np.where(adata.var['var_names'] == gene_name)[0][0]
    gene_acc = adata.layers['acceleration'][:,gene_index].tolist()
    adata.obs[gene_name+'_accel'] = gene_acc
    return adata

In [20]:
def gene_curvature(adata, gene_name):
    gene_index = np.where(adata.var['var_names'] == gene_name)[0][0]
    gene_cur = adata.layers['curvature'][:,gene_index].tolist()
    adata.obs[gene_name+'_curv'] = gene_cur
    return adata

In [21]:
def gene_velocity(adata, gene_name):
    gene_index = np.where(adata.var['var_names'] == gene_name)[0][0]
    gene_vel = adata.layers['velocity_S'][:,gene_index].toarray().flatten().tolist()
    adata.obs[gene_name+'_vel'] = gene_vel
    return adata

In [22]:
def gene_exp(adata, gene_name):
    gene_index = np.where(adata.var['var_names'] == gene_name)[0][0]
    gene_exp = adata.X[:,gene_index].toarray().flatten().tolist()
    adata.obs[gene_name+'_gene_exp'] = gene_exp
    return adata

In [23]:
def connect_ind_same_UMAP(adata_ind, adata_same, ind_stable_points, same_stable_points, fileName):
    dose = np.unique(adata_ind.obs['dosage'])[0]
    print("Dose ", dose, ":")
    
    new_coords = []
    
    plt.figure(figsize=(15,3))
    plt.subplot(1, 3, 1)
    plt.scatter(adata_same.obsm['X_umap'][:,0], adata_same.obsm['X_umap'][:,1], c="lightgrey")
    dyn.pl.zstreamline(adata_same)
    plt.scatter(same_stable_points[:,0], same_stable_points[:,1], c='r', label='stable')
    plt.title("Same UMAP ref - original fp")
    
    plt.subplot(1, 3, 2)
    plt.scatter(adata_same.obsm['X_umap'][:,0], adata_same.obsm['X_umap'][:,1], c="lightgrey")
    dyn.pl.zstreamline(adata_same)
    
    # Plotting the individual fixed points onto the same UMAP reference
    for i in range(len(ind_stable_points)):
        index = getNNeighbors([ind_stable_points[i][0], ind_stable_points[i][1]], adata_ind.obsm['X_umap'], 1)[0][0]
        fp_coords = ind_stable_points[i]
        ind_coords = adata_ind.obsm['X_umap'][index]
        dist = np.sqrt((fp_coords[0] - ind_coords[0])**2 + (fp_coords[1] - ind_coords[1])**2)
        if dist < 1:
            new_coords.append(ind_coords.tolist())
            plt.scatter(adata_same.obsm['X_umap'][index][0], adata_same.obsm['X_umap'][index][1], label="stable point", s=60)
    #plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1))
    plt.title("Ind fixed points mapped onto same UMAP ref")
    
    # Plotting the same UMAP reference onto the individual fixed points
    plt.subplot(1, 3, 3)
    plt.scatter(adata_ind.obsm['X_umap'][:,0], adata_ind.obsm['X_umap'][:,1], c="lightgrey")
    dyn.pl.zstreamline(adata_ind)
    for i in range(len(new_coords)):
        plt.scatter(new_coords[i][0], new_coords[i][1], label="stable point", s=60)
    #plt.legend(loc="upper right", bbox_to_anchor=(1.3, 1))
    plt.title("Ind fixed points mapped onto ind UMAP ref")
    plt.savefig(fileName+"mapping_"+str(dose), dpi=300)
    plt.show()

In [24]:
def connect_ind_same_UMAP_overlap(adata, adata_ind, adata_same, ind_stable_points, same_stable_points, fileName):
    dose = np.unique(adata_ind.obs['dosage'])[0]
    print("Dose ", dose, ":")
    
    new_coords = [] #filtered fixed points for the individual UMAP ref 
    
    # Plots the original fixed points on the same UMAP ref
    plt.figure(figsize=(15,4))
    plt.subplot(1, 3, 1)
    plt.scatter(adata.obsm['X_umap'][:,0], adata.obsm['X_umap'][:,1], c = 'lavender')
    plt.scatter(adata_same.obsm['X_umap'][:,0], adata_same.obsm['X_umap'][:,1], c="silver", alpha=0.4)
    dyn.pl.zstreamline(adata_same)
    plt.scatter(same_stable_points[:,0], same_stable_points[:,1], c='r', label='stable')
    #plt.title("Same UMAP ref - original fp")
    
    #Plots the overlapping fixed points from the ind and same ref as bigger dots 
    #Plots non overlapping fixed points as smaller dots
    plt.subplot(1, 3, 2)
    plt.scatter(adata.obsm['X_umap'][:,0], adata.obsm['X_umap'][:,1], c = 'lavender')
    plt.scatter(adata_same.obsm['X_umap'][:,0], adata_same.obsm['X_umap'][:,1], c="silver", alpha=0.4)
    dyn.pl.zstreamline(adata_same)
    
    mapped_points = [] #tracks the mapped fixed points ind --> same UMAP ref
    # Plots the individual fixed points onto the same UMAP reference
    for i in range(len(ind_stable_points)):
        index = getNNeighbors([ind_stable_points[i][0], ind_stable_points[i][1]], adata_ind.obsm['X_umap'], 1)[0][0]
        fp_coords = ind_stable_points[i]
        ind_coords = adata_ind.obsm['X_umap'][index]
        dist = np.sqrt((fp_coords[0] - ind_coords[0])**2 + (fp_coords[1] - ind_coords[1])**2)
        if dist < 1:
            new_coords.append(ind_coords.tolist())
            #print([adata_same.obsm['X_umap'][index][0], adata_same.obsm['X_umap'][index][1]])
            mapped_points.append([adata_same.obsm['X_umap'][index][0], adata_same.obsm['X_umap'][index][1]])
            plt.scatter(adata_same.obsm['X_umap'][index][0], adata_same.obsm['X_umap'][index][1], label="stable point", s=60)
    
    to_keep = [] #fixed points from the same UMAP ref that are inside the boundary 
    #Filters out fixed points from the same UMAP ref that are outside the boundary
    for i in range(len(same_stable_points)):
        index = getNNeighbors([same_stable_points[i][0], same_stable_points[i][1]], adata_same.obsm['X_umap'], 1)[0][0]
        fp_coords = same_stable_points[i]
        ind_coords = adata_same.obsm['X_umap'][index]
        dist = np.sqrt((fp_coords[0] - ind_coords[0])**2 + (fp_coords[1] - ind_coords[1])**2)
        if dist < 1:
            to_keep.append(fp_coords[0:2].tolist())
            plt.scatter(adata_same.obsm['X_umap'][index][0], adata_same.obsm['X_umap'][index][1], c='r', s=60)
    #plt.title("Ind + same fp mapped onto same UMAP ref")

    plt.subplot(1, 3, 3)
    plt.scatter(adata.obsm['X_umap'][:,0], adata.obsm['X_umap'][:,1], c = 'lavender')
    plt.scatter(adata_same.obsm['X_umap'][:,0], adata_same.obsm['X_umap'][:,1], c="silver", alpha=0.4)
    dyn.pl.zstreamline(adata_same)
    
    for i in range(len(mapped_points)):
        plt.scatter(mapped_points[i][0], mapped_points[i][1], label="stable point", s=60, c='b')
    
    # checks if ind + same UMAP fixed points overlap - plots as a bigger point
    # plots the non overlapping fixed points as smaller points
    for i in range(len(to_keep)):
        index = getNNeighbors(to_keep[i], mapped_points, 1)[0][0]
        cur_coord = to_keep[i]
        other_coord = mapped_points[index]
        dist = np.sqrt((cur_coord[0] - other_coord[0])**2 + (cur_coord[1] - other_coord[1])**2)
        if dist < 2:
            plt.scatter(cur_coord[0], cur_coord[1], label="stable point", s=300, c='greenyellow')
        else:
            plt.scatter(cur_coord[0], cur_coord[1], label="stable point", s=60, c='r')
    #print(mapped_points)  
    #for i in range(len(mapped_points)):
    #    plt.scatter(mapped_points[i][0], mapped_points[i][1], label="stable point", s=60, c='b')
        
    #plt.title("Ind + same fp mapped onto same UMAP ref")
    
    
    # Plotting the same UMAP reference onto the individual fixed points
    #plt.subplot(2, 2, 4)
    #plt.scatter(adata_ind.obsm['X_umap'][:,0], adata_ind.obsm['X_umap'][:,1], c="silver", alpha=0.4)
    #dyn.pl.zstreamline(adata_ind)
    #for i in range(len(new_coords)):
    #    plt.scatter(new_coords[i][0], new_coords[i][1], label="stable point", s=60)
    #plt.title("Ind fixed points mapped onto ind UMAP ref")
    #plt.savefig(fileName+"mapping_overlapping_"+str(dose), dpi=300)
    plt.show()

In [25]:
def connect_ind_same_UMAP_overlap2(adata, adata_ind, adata_same, ind_stable_points, same_stable_points, fileName):
    dose = np.unique(adata_ind.obs['dosage'])[0]
    print("Dose ", dose, ":")
    
    new_coords = [] #filtered fixed points for the individual UMAP ref 
    
    # Plots the original fixed points on the same UMAP ref
#     plt.figure(figsize=(15,4))
#     plt.subplot(1, 3, 1)
#     plt.scatter(adata.obsm['X_umap'][:,0], adata.obsm['X_umap'][:,1], c = 'lavender')
#     plt.scatter(adata_same.obsm['X_umap'][:,0], adata_same.obsm['X_umap'][:,1], c="silver", alpha=0.4)
#     dyn.pl.zstreamline(adata_same)
#     plt.scatter(same_stable_points[:,0], same_stable_points[:,1], c='r', label='stable')
#     #plt.title("Same UMAP ref - original fp")
    
    #Plots the overlapping fixed points from the ind and same ref as bigger dots 
    #Plots non overlapping fixed points as smaller dots
    #plt.subplot(1, 3, 2)
#     plt.scatter(adata.obsm['X_umap'][:,0], adata.obsm['X_umap'][:,1], c = 'lavender')
#     plt.scatter(adata_same.obsm['X_umap'][:,0], adata_same.obsm['X_umap'][:,1], c="silver", alpha=0.4)
#     dyn.pl.zstreamline(adata_same)
    
    mapped_points = [] #tracks the mapped fixed points ind --> same UMAP ref
    # Plots the individual fixed points onto the same UMAP reference
    for i in range(len(ind_stable_points)):
        index = getNNeighbors([ind_stable_points[i][0], ind_stable_points[i][1]], adata_ind.obsm['X_umap'], 1)[0][0]
        fp_coords = ind_stable_points[i]
        ind_coords = adata_ind.obsm['X_umap'][index]
        dist = np.sqrt((fp_coords[0] - ind_coords[0])**2 + (fp_coords[1] - ind_coords[1])**2)
        if dist < 1:
            new_coords.append(ind_coords.tolist())
            #print([adata_same.obsm['X_umap'][index][0], adata_same.obsm['X_umap'][index][1]])
            mapped_points.append([adata_same.obsm['X_umap'][index][0], adata_same.obsm['X_umap'][index][1]])
            #plt.scatter(adata_same.obsm['X_umap'][index][0], adata_same.obsm['X_umap'][index][1], label="stable point", s=60)
    
    to_keep = [] #fixed points from the same UMAP ref that are inside the boundary 
    #Filters out fixed points from the same UMAP ref that are outside the boundary
    for i in range(len(same_stable_points)):
        index = getNNeighbors([same_stable_points[i][0], same_stable_points[i][1]], adata_same.obsm['X_umap'], 1)[0][0]
        fp_coords = same_stable_points[i]
        ind_coords = adata_same.obsm['X_umap'][index]
        dist = np.sqrt((fp_coords[0] - ind_coords[0])**2 + (fp_coords[1] - ind_coords[1])**2)
        if dist < 1:
            to_keep.append(fp_coords[0:2].tolist())
            #plt.scatter(adata_same.obsm['X_umap'][index][0], adata_same.obsm['X_umap'][index][1], c='r', s=60)
    #plt.title("Ind + same fp mapped onto same UMAP ref")

    #plt.subplot(1, 3, 3)
    plt.scatter(adata.obsm['X_umap'][:,0], adata.obsm['X_umap'][:,1], c = 'lavender')
    plt.scatter(adata_same.obsm['X_umap'][:,0], adata_same.obsm['X_umap'][:,1], c="silver", alpha=0.4)
    dyn.pl.zstreamline(adata_same)
    
    for i in range(len(mapped_points)):
        plt.scatter(mapped_points[i][0], mapped_points[i][1], label="stable point", s=60, c='b')
    
    # checks if ind + same UMAP fixed points overlap - plots as a bigger point
    # plots the non overlapping fixed points as smaller points
    for i in range(len(to_keep)):
        index = getNNeighbors(to_keep[i], mapped_points, 1)[0][0]
        cur_coord = to_keep[i]
        other_coord = mapped_points[index]
        dist = np.sqrt((cur_coord[0] - other_coord[0])**2 + (cur_coord[1] - other_coord[1])**2)
        if dist < 2:
            plt.scatter(cur_coord[0], cur_coord[1], label="stable point", s=300, c='greenyellow')
        else:
            plt.scatter(cur_coord[0], cur_coord[1], label="stable point", s=60, c='r')
    #print(mapped_points)  
    #for i in range(len(mapped_points)):
    #    plt.scatter(mapped_points[i][0], mapped_points[i][1], label="stable point", s=60, c='b')
        
    #plt.title("Ind + same fp mapped onto same UMAP ref")
    
    
    # Plotting the same UMAP reference onto the individual fixed points
    #plt.subplot(2, 2, 4)
    #plt.scatter(adata_ind.obsm['X_umap'][:,0], adata_ind.obsm['X_umap'][:,1], c="silver", alpha=0.4)
    #dyn.pl.zstreamline(adata_ind)
    #for i in range(len(new_coords)):
    #    plt.scatter(new_coords[i][0], new_coords[i][1], label="stable point", s=60)
    #plt.title("Ind fixed points mapped onto ind UMAP ref")
    plt.savefig(fileName+"mapping_overlapping_"+str(dose), dpi=300)
    plt.show()

## Plotting trajectory and gene expression

In [3]:
# Using the start and end coords the closest cell is obtained
# LAP is performed and the resulting trajectory is plotted on the PCA reference space

def plot_single_trajectory(start_coord, end_coord, adata, num_points):
    start_index = getNNeighbors(start_coord, adata.obsm['X_pca'][:,0:2], k=1)
    end_index = getNNeighbors(end_coord, adata.obsm['X_pca'][:,0:2], k=1)

    start_cell = adata.obsm['X_pca'][start_index[0][0]]
    end_cell = adata.obsm['X_pca'][end_index[0][0]]

    adata.obsm['X_pca'] = adata.obsm['X_pca'][:,0:2]
    adata.uns['PCs'] = adata.uns['PCs'][:,0:2]

    dyn.pd.least_action(adata, start_index[0], end_index[0], n_points=num_points)
    
    path_sol = adata.uns['LAP_pca']['prediction'][0]
    
    plt.scatter(adata.obsm["X_pca"][:,0], adata.obsm["X_pca"][:,1], color = "lightgrey")
    plt.scatter(path_sol[:,0], path_sol[:,1], c=path_sol[:,0])
    plt.scatter(start_cell[0], start_cell[1], marker = "*", color = "red", label="start")
    plt.scatter(end_cell[0], end_cell[1], marker = "*", color = "yellow", label = "end")
    plt.legend(loc="upper left")
    plt.show()

In [4]:
def pca_to_expr(X, PCs, mean=0, func=None):
    # reverse project from PCA back to raw expression space
    if PCs.shape[1] == X.shape[1]:
        exprs = X @ PCs.T + mean
        if func is not None:
            exprs = func(exprs)
    else:
        raise Exception("PCs dim 1 (%d) does not match X dim 1 (%d)." % (PCs.shape[1], X.shape[1]))
    return exprs

In [5]:
def group_score(adata, expr_output, all_genes, gene_list: list):

    tmp = all_genes.intersection(gene_list)
    #print(tmp.shape)
    # use indices
    intersect_genes = [all_genes.get_loc(i) for i in tmp]
    #print(intersect_genes)

    if len(intersect_genes) == 0:
        raise Exception(f"your adata doesn't have any gene from the gene_list {gene_list}.")

    #print(intersect_genes)
    expression_matrix = expr_output[:, intersect_genes]
    expression_matrix = log1p_(adata, expression_matrix)
    
    if issparse(expression_matrix):
        expression_matrix.data = np.log1p(expression_matrix.data)
        scores = expression_matrix.sum(1).A1
    else:
        scores = np.log1p(expression_matrix).sum(1)

    scores = (scores - scores.mean()) / scores.std()

    return scores

In [6]:
def batch_group_score(adata, expr_output, all_genes, gene_lists: list):
    batch_scores = collections.OrderedDict()
    for gene_list in gene_lists:
        batch_scores[gene_list] = group_score(adata, expr_output, all_genes, gene_lists[gene_list])
    return batch_scores

In [7]:
def log1p_(adata, X_data):
    if "norm_method" not in adata.uns["pp"].keys():
        return X_data
    else:
        if adata.uns["pp"]["norm_method"] is None:
            if issparse(X_data):
                X_data.data = np.log1p(X_data.data)
            else:
                X_data = np.log1p(X_data)

        return X_data

In [8]:
def get_LAP(start_coord, end_coord, adata, num_points, dim=30):
    #start_index = getNNeighbors(start_coord, adata.obsm['X_pca'][:,dim], k=1)
    #end_index = getNNeighbors(end_coord, adata.obsm['X_pca'][:,dim], k=1)

    #start_cell = adata.obsm['X_pca'][start_index[0][0]]
    #end_cell = adata.obsm['X_pca'][end_index[0][0]]

    adata.obsm['X_pca'] = adata.obsm['X_pca'][:,:dim]
    adata.uns['PCs'] = adata.uns['PCs'][:,:dim]

    #traj = dyn.pd.least_action(adata, start_index[0], end_index[0], n_points=num_points)
    traj = dyn.pd.least_action(adata, start_coord, end_coord, n_points=num_points)
    
    path_sol = adata.uns['LAP_pca']['prediction'][0]
    
    x_pca = adata.uns['LAP_pca']['prediction'][0]
    PCs = adata.uns['PCs']
    mean = adata.uns['pca_mean']
    #expr_output = pca_to_expr(x_pca, PCs, mean, np.expm1)
    expr_output = pca_to_expr(x_pca, PCs, mean, None)
    
    all_genes = adata.var.index[np.where(adata.var['use_for_dynamics'] == True)[0]]
    #display(pd.DataFrame(all_genes))

    return path_sol, expr_output, all_genes, traj

In [7]:
# def plot_single_trajectory(start_coord, end_coord, adata):
#     path_sol = dyn.pd.least_action(adata, start_coord, end_coord)
#     plt.scatter(adata.obsm["X_umap"][:,0], adata.obsm["X_umap"][:,1], color = "lightgrey")
#     plt.scatter(path_sol[:,0], path_sol[:,1], c=path_sol[:,0])
#     plt.scatter(start_coord[0], start_coord[1], marker = "*", color = "red", label="start")
#     plt.scatter(end_coord[0], end_coord[1], marker = "*", color = "black", label = "end")
#     plt.legend(loc="upper left")
#     #plt.savefig('Plots/EMT_trajectory/DE_genes_top_path/top_trajectory', dpi=300)
#     plt.show()
#     return path_sol

In [ ]:
# Using the start and end coords the closest cell is obtained
# LAP is performed and the resulting trajectory is plotted on the PCA reference space
# Only varies the starting point - not the end point
def plot_multiple_trajectories(start_coord, end_coord, adata, number_of_cells):
    start_indices = getNNeighbors(start_coord, adata.obsm['X_umap'], k=number_of_cells)[0]
    #end_indices = getNNeighbors([8.5,1], adata.obsm['X_umap'], k=10)[0]
    
    for i in range(len(start_indices)):
        start = adata.obsm["X_umap"][start_indices[i], :]
        print("Starting point: ", start)
        plot_single_trajectory(start, end_coord, adata)
        #dyn.pd.least_action(adata, start, end_coord)
        #path_sol = adata.uns['LAP_pca']['prediction'][0]

#         plt.scatter(adata.obsm["X_pca"][:,0], adata.obsm["X_"][:,1], color = "lightgrey")
#         plt.scatter(path_sol[:,0], path_sol[:,1], c=path_sol[:,0])
#         plt.scatter(adata.obsm["X_umap"][start_indices[i],0], adata.obsm["X_umap"][start_indices[i],1], marker = "*", color = "red", label="start")
#         #plt.scatter(adata.obsm["X_umap"][end_coord,0], adata.obsm["X_umap"][end_coord,1], marker = "*", color = "black", label = "end")
#         plt.scatter(end_coord[0], end_coord[1], marker = "*", color = "black", label = "end")
#         #plt.title("TGFB dose 12.5 - least action path")
#         plt.legend(loc="upper left")
#         filename = "Plots/EMT_trajectory/Trajectories/traj_"+str(i)
#         plt.savefig(filename, dpi=300)
#         plt.show()

In [11]:
# Parameters: 
#     num_points: number of points for each trajectory
#     num_traj: number of trajectories to generate
# Returns:
#     pathways: 2D list containing the coordinates for each trajectory
#     progression: 2D list containing the cell cycle categories for each trajectory

def cell_progression(start_coord, end_coord, adata, num_points, num_traj, genes):
    phases = np.array(['G1-S', 'S', 'G2-M', 'M', 'M-G1'], dtype=object)
    myColors = ((255/256,91/256,165/256),(219/256,73/256,172/256), (176/256,124/256,218/256), (107/256,167/256,214/256),(100/256,212/256,199/256))
    progression = []
    pathways = []
    expr = []
    genes_used = []
    trajs = []
    
    # Using k nearest points from the given start coordinate
    start_points = getNNeighbors(start_coord, adata.obsm['X_pca'][:,0:2], k=num_traj)[0]
    cell_phase_genes = get_cell_phase_genes(adata, None)
    
    # Obtains a trajectory --> cell phase score for each point in the trajectory
    for i in range(num_traj):
        path_sol, expr_output, all_genes, traj = get_LAP(adata.obsm['X_pca'][:,0:2][start_points[i]], end_coord, adata, num_points)
        pathways.append(path_sol)
        trajs.append(traj)
        phase_scores = pd.DataFrame(batch_group_score(adata, expr_output, all_genes, cell_phase_genes))
        phases = phase_scores.columns

        gene_indexes = []
        for j in range(len(genes)):
            if genes[j] in adata.uns['LAP_pca']['genes'].tolist():
                gene_indexes.append(all_genes.get_loc(genes[j]))
                if i == 0:
                    genes_used.append(genes[j])
        
        extracted_gene_expr = expr_output[:,gene_indexes]
        expr.append(extracted_gene_expr)
        
        final_num = []
        for i in range(len(phase_scores)):
            line = np.array(phase_scores.loc[i])
            line_max = np.max(line)
            phase_index = np.where(line == line_max)[0][0]
            final_num.append(phase_index)
        progression.append(final_num)

    pathways = np.array(pathways)
    top_prog, top_paths, bottom_prog, bottom_paths, top_index, bottom_index = separate_paths2(progression, pathways)
        
    height = 0
    for i in range(len(top_prog)):
        x_val = 0
        for j in range(len(top_prog[i])):
            plt.plot([x_val, x_val + 1], [height,height], color=myColors[top_prog[i][j]], linewidth=35)
            x_val += 1
        height += 1

    first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
    sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
    third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
    fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
    fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
    plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
    plt.ylabel("Cells")
    plt.xlabel("Top trajectories")
    plt.show()
    
    height = 0
    for i in range(len(bottom_prog)):
        x_val = 0
        for j in range(len(bottom_prog[i])):
            plt.plot([x_val, x_val + 1], [height,height], color=myColors[bottom_prog[i][j]], linewidth=60)
            x_val += 1
        height += 1

    first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
    sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
    third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
    fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
    fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
    plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
    plt.ylabel("Cells")
    plt.xlabel("Bottom trajectories")
    plt.show()
    
    # Plots each trajectories cell phase score in a line plot
#     height = 0
#     for i in range(len(progression)):
#         x_val = 0
#         for j in range(len(progression[i])):
#             plt.plot([x_val, x_val + 1], [height,height], color=myColors[progression[i][j]], linewidth=20)
#             x_val += 1
#         height += 1
#     first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
#     sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
#     third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
#     fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
#     fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
#     plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
#     plt.ylabel("Cells")
#     plt.xlabel("Trajectory")
#     plt.show()
    
    # Plots each trajectory for reference
#     for i in range(len(pathways)):
#         plt.scatter(adata.obsm["X_pca"][:,0], adata.obsm["X_pca"][:,1], color = "lightgrey")
#         plt.scatter(pathways[i][:,0], pathways[i][:,1])
#         plt.show()
#     plt.show()
    
    return top_prog, top_paths, bottom_prog, bottom_paths, expr, top_index, bottom_index, genes_used, trajs

In [2]:
# Parameters: 
#     num_points: number of points for each trajectory
#     num_traj: number of trajectories to generate
# Returns:
#     pathways: 2D list containing the coordinates for each trajectory
#     progression: 2D list containing the cell cycle categories for each trajectory

# Selects a bigger range of start points 

def cell_progression2(upper_left, lower_right, end_point, adata, num_points, num_traj, genes):
    phases = np.array(['G1-S', 'S', 'G2-M', 'M', 'M-G1'], dtype=object)
    myColors = ((255/256,91/256,165/256),(219/256,73/256,172/256), (176/256,124/256,218/256), (107/256,167/256,214/256),(100/256,212/256,199/256))
    progression = []
    pathways = []
    expr = []
    genes_used = []
    trajs = []
    
    # Using k nearest points from the given start coordinate
    start_points = []
    x_num = None
    y_num = None
    for i in range(num_traj):
        if upper_left[0] < lower_right[0]:
            x_num = np.random.uniform(upper_left[0], lower_right[0])
        else:
            x_num = np.random.uniform( lower_right[0], upper_left[0])
        
        if upper_left[1] < lower_right[1]:
            y_num = np.random.uniform(upper_left[1], lower_right[1])
        else:
            y_num = np.random.uniform( lower_right[1], upper_left[1])
            
        start_points.append([x_num, y_num])
        
    #start_points = getNNeighbors(start_coord, adata.obsm['X_pca'][:,0:2], k=num_traj)[0]
    
    cell_phase_genes = get_cell_phase_genes(adata, None)
    
    # Obtains a trajectory --> cell phase score for each point in the trajectory
    for i in range(num_traj):
        #path_sol, expr_output, all_genes = get_LAP(adata.obsm['X_pca'][:,0:2][start_points[i]], end_coord, adata, num_points)
        path_sol, expr_output, all_genes, traj = get_LAP(start_points[i], end_coord, adata, num_points)
        
        #print(expr_output)
        #print(start_points[i], end_coord)
        #plt.scatter(adata.obsm['X_pca'][:,0], adata.obsm['X_pca'][:,1])
        #plt.scatter(path_sol[:,0], path_sol[:,1])
        #plt.show()
        pathways.append(path_sol)
        trajs.append(traj)
        phase_scores = pd.DataFrame(batch_group_score(adata, expr_output, all_genes, cell_phase_genes))
        phases = phase_scores.columns

        gene_indexes = []
        for j in range(len(genes)):
            if genes[j] in adata.uns['LAP_pca']['genes'].tolist():
                gene_indexes.append(all_genes.get_loc(genes[j]))
                if i == 0:
                    genes_used.append(genes[j])
        
        extracted_gene_expr = expr_output[:,gene_indexes]
        expr.append(extracted_gene_expr)
        
        final_num = []
        #display(phase_scores[0:10])
        for i in range(len(phase_scores)):
            line = np.array(phase_scores.loc[i])
            line_max = np.max(line)
            phase_index = np.where(line == line_max)[0][0]
            final_num.append(phase_index)
        progression.append(final_num)

    pathways = np.array(pathways)
    top_prog, top_paths, bottom_prog, bottom_paths, top_index, bottom_index = separate_paths2(progression, pathways)
        
    height = 0
    for i in range(len(top_prog)):
        x_val = 0
        for j in range(len(top_prog[i])):
            plt.plot([x_val, x_val + 1], [height,height], color=myColors[top_prog[i][j]], linewidth=35)
            x_val += 1
        height += 1

    first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
    sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
    third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
    fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
    fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
    plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
    plt.ylabel("Cells")
    plt.xlabel("Top trajectories")
    plt.show()
    
    height = 0
    for i in range(len(bottom_prog)):
        x_val = 0
        for j in range(len(bottom_prog[i])):
            plt.plot([x_val, x_val + 1], [height,height], color=myColors[bottom_prog[i][j]], linewidth=60)
            x_val += 1
        height += 1

    first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
    sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
    third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
    fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
    fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
    plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
    plt.ylabel("Cells")
    plt.xlabel("Bottom trajectories")
    plt.show()
    
    # Plots each trajectories cell phase score in a line plot
#     height = 0
#     for i in range(len(progression)):
#         x_val = 0
#         for j in range(len(progression[i])):
#             plt.plot([x_val, x_val + 1], [height,height], color=myColors[progression[i][j]], linewidth=20)
#             x_val += 1
#         height += 1
#     first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
#     sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
#     third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
#     fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
#     fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
#     plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
#     plt.ylabel("Cells")
#     plt.xlabel("Trajectory")
#     plt.show()
    
    # Plots each trajectory for reference
#     for i in range(len(pathways)):
#         plt.scatter(adata.obsm["X_pca"][:,0], adata.obsm["X_pca"][:,1], color = "lightgrey")
#         plt.scatter(pathways[i][:,0], pathways[i][:,1])
#         plt.show()
#     plt.show()
    
    return top_prog, top_paths, bottom_prog, bottom_paths, expr, top_index, bottom_index, genes_used, trajs

In [ ]:
# Parameters: 
#     num_points: number of points for each trajectory
#     num_traj: number of trajectories to generate
# Returns:
#     pathways: 2D list containing the coordinates for each trajectory
#     progression: 2D list containing the cell cycle categories for each trajectory

# Selects a bigger range of start points and end points

def cell_progression3(upper_left, lower_right, upper_left2, lower_right2, adata, num_points, num_traj, genes):
    phases = np.array(['G1-S', 'S', 'G2-M', 'M', 'M-G1'], dtype=object)
    myColors = ((255/256,91/256,165/256),(219/256,73/256,172/256), (176/256,124/256,218/256), (107/256,167/256,214/256),(100/256,212/256,199/256))
    progression = []
    pathways = []
    expr = []
    genes_used = []
    trajs = []
    
    # Using k nearest points from the given start coordinate
    start_points = []
    end_points = []
    x_num = None
    y_num = None
    x_num2 = None
    y_num2 = None
    for i in range(num_traj):
        if upper_left[0] < lower_right[0]:
            x_num = np.random.uniform(upper_left[0], lower_right[0])
        else:
            x_num = np.random.uniform( lower_right[0], upper_left[0])
        
        if upper_left[1] < lower_right[1]:
            y_num = np.random.uniform(upper_left[1], lower_right[1])
        else:
            y_num = np.random.uniform( lower_right[1], upper_left[1])
        
        if upper_left2[0] < lower_right2[0]:
            x_num2 = np.random.uniform(upper_left2[0], lower_right2[0])
        else:
            x_num2 = np.random.uniform( lower_right2[0], upper_left2[0])
        
        if upper_left2[1] < lower_right2[1]:
            y_num2 = np.random.uniform(upper_left2[1], lower_right2[1])
        else:
            y_num2 = np.random.uniform( lower_right2[1], upper_left2[1])
        
        start_points.append([x_num, y_num])
        end_points.append([x_num2, y_num2])
        
        
    #start_points = getNNeighbors(start_coord, adata.obsm['X_pca'][:,0:2], k=num_traj)[0]
    
    cell_phase_genes = get_cell_phase_genes(adata, None)
    
    # Obtains a trajectory --> cell phase score for each point in the trajectory
    for i in range(num_traj):
        #path_sol, expr_output, all_genes = get_LAP(adata.obsm['X_pca'][:,0:2][start_points[i]], end_coord, adata, num_points)
        path_sol, expr_output, all_genes, traj = get_LAP(start_points[i], end_points[i], adata, num_points)
        pathways.append(path_sol)
        trajs.append(traj)
        phase_scores = pd.DataFrame(batch_group_score(adata, expr_output, all_genes, cell_phase_genes))
        phases = phase_scores.columns

        gene_indexes = []
        for j in range(len(genes)):
            if genes[j] in adata.uns['LAP_pca']['genes'].tolist():
                gene_indexes.append(all_genes.get_loc(genes[j]))
                if i == 0:
                    genes_used.append(genes[j])
        
        extracted_gene_expr = expr_output[:,gene_indexes]
        expr.append(extracted_gene_expr)
        
        final_num = []
        #display(phase_scores)
        for i in range(len(phase_scores)):
            line = np.array(phase_scores.loc[i])
            line_max = np.max(line)
            #print(line, line_max)
            phase_index = np.where(line == line_max)[0][0]
            final_num.append(phase_index)
        progression.append(final_num)

    pathways = np.array(pathways)
    top_prog, top_paths, bottom_prog, bottom_paths, top_index, bottom_index = separate_paths2(progression, pathways)
        
    height = 0
    for i in range(len(top_prog)):
        x_val = 0
        for j in range(len(top_prog[i])):
            plt.plot([x_val, x_val + 1], [height,height], color=myColors[top_prog[i][j]], linewidth=35)
            x_val += 1
        height += 1

    first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
    sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
    third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
    fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
    fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
    plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
    plt.ylabel("Cells")
    plt.xlabel("Top trajectories")
    plt.show()
    
    height = 0
    for i in range(len(bottom_prog)):
        x_val = 0
        for j in range(len(bottom_prog[i])):
            plt.plot([x_val, x_val + 1], [height,height], color=myColors[bottom_prog[i][j]], linewidth=60)
            x_val += 1
        height += 1

    first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
    sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
    third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
    fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
    fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
    plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
    plt.ylabel("Cells")
    plt.xlabel("Bottom trajectories")
    plt.show()
    
    # Plots each trajectories cell phase score in a line plot
#     height = 0
#     for i in range(len(progression)):
#         x_val = 0
#         for j in range(len(progression[i])):
#             plt.plot([x_val, x_val + 1], [height,height], color=myColors[progression[i][j]], linewidth=20)
#             x_val += 1
#         height += 1
#     first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
#     sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
#     third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
#     fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
#     fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
#     plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
#     plt.ylabel("Cells")
#     plt.xlabel("Trajectory")
#     plt.show()
    
    # Plots each trajectory for reference
#     for i in range(len(pathways)):
#         plt.scatter(adata.obsm["X_pca"][:,0], adata.obsm["X_pca"][:,1], color = "lightgrey")
#         plt.scatter(pathways[i][:,0], pathways[i][:,1])
#         plt.show()
#     plt.show()
    
    return top_prog, top_paths, bottom_prog, bottom_paths, expr, top_index, bottom_index, genes_used, trajs

In [30]:
# def plot_trajectory(start_coord, end_coord, adata, number_of_cells):
    
#     start_indices = getNNeighbors(start_coord, adata.obsm['X_umap'], k=number_of_cells)[0]
#     #end_indices = getNNeighbors([8.5,1], adata.obsm['X_umap'], k=10)[0]
    
#     for i in range(len(start_indices)):
#         start = adata.obsm["X_umap"][start_indices[i], :]
#         print("Starting point: ", start)
#         print("End point: ", end_coord)
#         path_sol = dyn.pd.least_action(adata, start, end_coord)
#         print(len(path_sol))

#         plt.scatter(adata.obsm["X_umap"][:,0], adata.obsm["X_umap"][:,1], color = "lightgrey")
#         plt.scatter(path_sol[:,0], path_sol[:,1], c=path_sol[:,0])
#         plt.scatter(adata.obsm["X_umap"][start_indices[i],0], adata.obsm["X_umap"][start_indices[i],1], marker = "*", color = "red", label="start")
#         #plt.scatter(adata.obsm["X_umap"][end_coord,0], adata.obsm["X_umap"][end_coord,1], marker = "*", color = "black", label = "end")
#         plt.scatter(end_coord[0], end_coord[1], marker = "*", color = "black", label = "end")
#         #plt.title("TGFB dose 12.5 - least action path")
#         plt.legend(loc="upper left")
#         filename = "Plots/EMT_trajectory/Trajectories/traj_"+str(i)
#         plt.savefig(filename, dpi=300)
#         plt.show()

In [9]:
def finding_gene_names(gene_list, adata):
    indexes = []
    gene_names = []
    for i in range(len(gene_list)):
        gene_index = np.where(adata.var['var_names'] == gene_list[i])[0]
        if gene_index.shape[0] > 0:
            indexes.append(gene_index[0])
            gene_names.append(gene_list[i])
    gene_names = np.array(gene_names)
    return gene_names, indexes

In [49]:
## BUGS ##
def get_gene_progression(path_sol, adata, gene_names, indexes, groups):
    gene_progression = []
    interval = 1/groups
    
    gene_max = np.max(adata.X.toarray)
    gene_min = np.min(adata.X.toarray)
    
    for i in range(0,len(path_sol),1):
        indices = getNNeighbors(path_sol[i], adata.obsm['X_umap'], k=100)[0]

        temp = adata[indices,indexes]
        gene_exp = temp.X.toarray()

        mean = np.mean(gene_exp, axis = 0)
        order = np.argsort(mean)
        gene_exp2 = gene_exp[:, order]

        #num = np.arange(0,len(mean))
        
        total = interval
        groupings = []
        while(total<=1):
            if total == 1:
                groupings.append(np.where(mean >= np.quantile(mean, total - interval))[0].tolist())
            else:
                groupings.append(np.where((mean < np.quantile(mean,total)) & (mean >= np.quantile(mean, total - interval)))[0].tolist())
            total = total + interval
        temp2 = []
        for j in range(len(groupings)):
            temp2.append(gene_names[groupings[j]].tolist())
        gene_progression.append(temp2)
        
        #group1 = np.where(mean < np.quantile(mean, 0.33))[0]
        #group3 = np.where(mean > np.quantile(mean, 0.66))[0]
        #group2 = list(set(num.tolist()) - set(group1.tolist()))
        #group2 = list(set(group2) - set(group3.tolist()))

        #gene_progression.append([gene_names[group1].tolist(), gene_names[group2].tolist(), gene_names[group3].tolist()])

        #ax = sns.heatmap(gene_exp2, xticklabels=gene_names[order])
        #plt.yticks(rotation=360)
        #plt.xticks(rotation=45)
        #plt.savefig("Plots/EMT_trajectory/1000_genes/trajectory_"+str(i), dpi=300)
        #plt.show()
    return gene_progression

In [151]:
def get_gene_progression2(path_sol, adata, gene_names, indexes, groups):
    #Looping through all the genes
    genes_progression = []
    genes_quantiles = []
    for i in range(gene_names.shape[0]):
        #print(gene_names[i])
        exp = []
        #Looping through cells along the trajectory path
        for j in range(len(path_sol)):
            indices = getNNeighbors(path_sol[j], adata.obsm['X_umap'], k=20)[0]
            temp = adata[indices,indexes[i]]
            gene_exp = temp.X.toarray()
            exp.append(gene_exp)
        interval = 1/groups
        quantiles = np.arange(0, 1.01, interval)
        #for k in range(len(quantiles)):
        #    print(round(quantiles[k],3), "\t", np.quantile(exp, quantiles[k]))
        #print(quantiles)
        #print(i, ":", len(exp))
        if np.quantile(exp, quantiles[groups]) == 0:
            genes_progression.append(np.repeat([0], len(exp)))
            genes_quantiles.append(np.quantile(exp, quantiles))
        else:
            gene_prog = []
            for j in range(len(path_sol)):
                indices = getNNeighbors(path_sol[j], adata.obsm['X_umap'], k=20)[0]
                temp = adata[indices,indexes[i]]
                exp_mean = np.mean(temp.X.toarray())
                for k in range(1,len(quantiles)):
                    if (exp_mean >= np.quantile(exp, quantiles[k-1])) & (exp_mean < np.quantile(exp, quantiles[k])):
                        gene_prog.append(k-1)
            genes_progression.append(gene_prog)
            genes_quantiles.append(np.quantile(exp, quantiles))
    print(len(genes_progression))
    print(len(genes_quantiles))
    return genes_progression, genes_quantiles

In [146]:
def plot_gene_progression2(gene_names, progression, gene_quantiles, groups, filename, save=True):
    timepoints = np.arange(len(progression[0]))
    for i in range(len(progression)):
        #plt.plot(timepoints, progression[i], marker = 'o')
        #z = np.polyfit(timepoints, progression[i], 2)
        #p = np.poly1d(z)
        print(len(progression[i]))
        plt.bar(timepoints, progression[i], color=sns.color_palette("rocket", len(timepoints)), edgecolor='black', width=0.7)
        #plt.plot(timepoints, p(timepoints), 'r--')
        
        plt.title(gene_names[i])
        plt.ylim(-0.5, groups+0.5-1)
        #file = "Plots/EMT_trajectory/" + filename + "/" + gene_names[i] + "_bar_progression"
        file = filename + "/" + gene_names[i] + "_bar_progression"
        #print(gene_quantiles[i])
        plt.xlabel("Group 0: " + str(round(gene_quantiles[i][0],2)) + "   -   " + "Group " + str(groups) + ": " + str(round(gene_quantiles[i][groups],2)))
        if save == True:
            plt.savefig(file, dpi=300)
        plt.tight_layout()
        plt.show()

In [150]:
def get_ge_along_traj(gene_names, path, adata, groups, directory, save):
    parent_dir = "/Users/sophiahu/Documents/Xing lab/MCF10A_data_NEW/notebooks/vs_notebooks/Plots/EMT_trajectory/"
    new_path = os.path.join(parent_dir, directory)
    if os.path.isdir(new_path) == False:
        os.mkdir(new_path)
    gene_names, indexes = finding_gene_names(gene_names, adata)
    gene_prog, gene_quantiles = get_gene_progression2(path, adata, gene_names, indexes, groups)
    #print(len(gene_prog))
    #print(len(gene_quantiles))
    plot_gene_progression2(gene_names, gene_prog, gene_quantiles, groups, new_path, save)

In [155]:
def plot_acceleration_traj(gene_names, indexes, path, adata):
    for i in range(len(gene_names)):
        accel_means = []
        for j in range(len(path_sol)):
            indices = getNNeighbors(path_sol[j], adata.obsm['X_umap'], k=20)[0]
            temp = adata.layers['acceleration'][indices, indexes[i]]
            accel_means.append(np.mean(temp))
        x_axis = np.arange(0, len(accel_means))
        plt.plot(x_axis, accel_means)
        plt.title(gene_names[i])
        plt.show()

In [17]:
def plot_cell_cycle_along_traj(path, adata):
    phases = np.array(['M-G1','G1-S', 'S','G2-M','M'], dtype=object)
    progression = []
    for i in range(len(path)):
        indices = getNNeighbors(path[i], adata.obsm['X_umap'], k=20)[0]
        temp = adata[indices, :]
        phase_tracker = []
        for j in range(phases.shape[0]):
            phase_tracker.append(len(np.where(temp.obs['cell_cycle_phase'] == phases[j])[0]))
        max_phase = max(phase_tracker)
        phase = np.where(np.array(phase_tracker) == max_phase)[0][0]
        progression.append(phase)

    phases = phases.tolist()
    plt.bar(np.arange(0,len(path),1).tolist(), progression)
    plt.ylim(0,4.5)
    plt.show()

In [16]:
def get_cell_cycle_along_traj(path, adata):
    phases = np.array(['M-G1','G1-S', 'S','G2-M','M'], dtype=object)
    progression = []
    for i in range(path.shape[0]):
        indices = getNNeighbors(path[i], adata.obsm['X_umap'], k=20)[0]
        temp = adata[indices, :]
        phase_tracker = []
        for j in range(phases.shape[0]):
            phase_tracker.append(len(np.where(temp.obs['cell_cycle_phase'] == phases[j])[0]))
        max_phase = max(phase_tracker)
        phase = np.where(np.array(phase_tracker) == max_phase)[0][0]
        progression.append(phase)
    return progression

In [120]:
# Getting cell cycle phase for a single cell instead of averaging it
def get_cell_cycle_along_traj2(path, adata):
    phases = np.array(['M-G1','G1-S', 'S','G2-M','M'], dtype=object)
    progression = []
    for i in range(len(path)):
        index = getNNeighbors(path[i], adata.obsm['X_umap'], k=1)[0]
        temp = adata[index, :]
        phase = np.where(phases == temp.obs['cell_cycle_phase'][0])[0][0]
        progression.append(phase)
    return progression

In [21]:
def multiple_traj_cell_cycle_progression(start_coord, end_coord, adata, cells):
    indices = getNNeighbors(start_coord, adata.obsm['X_umap'], k=cells)[0] 
    cell_progressions = []
    paths = []
    for i in range(len(indices)):
        path_sol = dyn.pd.least_action(adata, adata.obsm['X_umap'][indices[i],:], end_coord)
        paths.append(path_sol)
        to_add = get_cell_cycle_along_traj(path_sol, adata)
        print(len(to_add))
        cell_progressions.append(to_add)
    return cell_progressions, np.array(paths)

In [112]:
# Plot trajectories with very different start points 
def multiple_traj_cell_cycle_progression2(top_left_corner, bottom_right_corner, end_coord, adata, cells):
    cell_progressions = []
    paths = []
    for i in range(cells):
        x_val = np.random.uniform(top_left_corner[0], bottom_right_corner[0])
        y_val = np.random.uniform(bottom_right_corner[1], top_left_corner[1])

        index = getNNeighbors([x_val, y_val], adata.obsm['X_umap'], k=1)[0]
        path_sol = dyn.pd.least_action(adata, adata.obsm['X_umap'][index,:][0], end_coord)
        paths.append(path_sol)
        to_add = get_cell_cycle_along_traj(path_sol, adata)
        print(len(to_add))
        cell_progressions.append(to_add)
    return cell_progressions, np.array(paths)

In [116]:
# Plot trajectories with very different start points 
def multiple_traj_cell_cycle_progression3(top_left_corner, bottom_right_corner, end_coord, adata, cells):
    cell_progressions = []
    paths = []
    count = 0
    #for i in range(cells):
    while count < cells:
        x_val = np.random.uniform(top_left_corner[0], bottom_right_corner[0])
        y_val = np.random.uniform(bottom_right_corner[1], top_left_corner[1])
        index = getNNeighbors([x_val, y_val], adata.obsm['X_umap'], k=1)[0]
        path_sol = dyn.pd.least_action(adata, adata.obsm['X_umap'][index,:][0], end_coord)
        toAdd = True
        for j in range(len(paths)):
            dtw_distance, _ = fastdtw(path_sol[:,0], path_sol[:,1], dist=euclidean)
            if dtw_distance < 15:
                toAdd = False
        if toAdd == True:
            paths.append(path_sol)
            to_add = get_cell_cycle_along_traj(path_sol, adata)
            print(len(to_add))
            cell_progressions.append(to_add)
            count += 1
    return cell_progressions, np.array(paths)

In [150]:
# Separates the trajectories as either coming from the top or bottom
def separate_paths(cell_prog, paths):
    top_paths = []
    bottom_paths = []
    top_path_cell_progressions = []
    bottom_path_cell_progressions = []
    top_index = []
    bottom_index = []
    
    for i in range(len(paths)):
        start_y = paths[i][0,1]
        length = paths[i].shape[0]

        #if np.where(paths[i][:,1] < start_y)[0].shape[0] > 3*(paths[i].shape[0]/4):
        if np.where(paths[i][int(1.5*(length/4)):int(3*(length/4)),1] < start_y)[0].shape[0] > 0:
            bottom_paths.append(paths[i])
            bottom_path_cell_progressions.append(cell_prog[i])
            bottom_index.append(i)
        else:
            top_paths.append(paths[i])
            top_path_cell_progressions.append(cell_prog[i])
            top_index.append(i)
            
    top_paths = np.array(top_paths)
    bottom_paths = np.array(bottom_paths)
    top_path_cell_progressions = np.array(top_path_cell_progressions)
    bottom_path_cell_progressions = np.array(bottom_path_cell_progressions)
    
    print(top_paths.shape[0])
    print(bottom_paths.shape[0])
    
    return top_path_cell_progressions, top_paths, bottom_path_cell_progressions, bottom_paths, top_index, bottom_index

In [ ]:
# Using the slope to determine if points are above or below the calculated slope
# Determines if a path is going along the top or bottom path
def separate_paths2(cell_prog, paths):
    top_paths = []
    bottom_paths = []
    top_path_cell_progressions = []
    bottom_path_cell_progressions = []
    top_index = []
    bottom_index = []
    
    for i in range(len(paths)):
        start_y = paths[i][0,1]
        start_x = paths[i][0,0]
        end_y = paths[i][len(paths[i])-1,1]
        end_x = paths[i][len(paths[i])-1,0]
        
        slope = (end_y - start_y)/(end_x - start_x)
        b = start_y - slope*start_x
        
        top = 0
        bottom = 0
        for j in range(len(paths[i])):
            y = paths[i][j,1]
            x = paths[i][j,0]
            if y > slope*x + b:
                top +=1
            elif y < slope*x + b:
                bottom += 1
        
        if top > bottom:
            top_paths.append(paths[i])
            top_path_cell_progressions.append(cell_prog[i])
            top_index.append(i)       
        else:
            bottom_paths.append(paths[i])
            bottom_path_cell_progressions.append(cell_prog[i])
            bottom_index.append(i)
            
    top_paths = np.array(top_paths)
    bottom_paths = np.array(bottom_paths)
    top_path_cell_progressions = np.array(top_path_cell_progressions)
    bottom_path_cell_progressions = np.array(bottom_path_cell_progressions)
    
    print(top_paths.shape[0])
    print(bottom_paths.shape[0])
    
    return top_path_cell_progressions, top_paths, bottom_path_cell_progressions, bottom_paths, top_index, bottom_index

In [113]:
def plot_cell_progression_lineplot(cell_progression, paths, intermediate_coord, line_width=10):
    interval_cell_progression = []
    cell_progression = np.array(cell_progression)
    path_len = []
    
    for j in range(paths.shape[0]):
#         x_coords = np.linspace(paths[j][0,0], paths[j][paths[j].shape[0]-1,0], paths[j].shape[0])
#         coords = np.vstack((paths[j][:,0], np.repeat(0, paths[j].shape[0]))).T
#         interval_coords = np.vstack((x_coords, np.repeat(0, x_coords.shape[0]))).T

#         temp = []
#         for i in range(interval_coords.shape[0]): 
#             index = getNNeighbors(interval_coords[i], coords, 1)[0][0]

#             temp.append(cell_progression[j][index])
#         interval_cell_progression.append(temp)
        path_len.append(paths[j].shape[0])

    # Plotting using multiple line plots
    import matplotlib.patches as mpatches

    myColors = ((248/256,177/256,149/256),(246/256,114/256,128/256),(192/256,108/256,132/256),(108/256,91/256,123/256),(53/256,92/256,125/256))

    first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
    sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
    third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
    fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
    fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
    plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))

    ax = plt.axes()
    ax.set_facecolor("white")

#     cell = 0
#     for i in range(len(interval_cell_progression)):
#         for j in range(len(interval_cell_progression[i])):
#             plt.plot([j, j+1], [cell, cell], color = myColors[interval_cell_progression[i][j]], linewidth=line_width)
#         cell = cell + 1
#     #plt.ylabel("Cells")
#     plt.xlabel("Trajectory")
#     plt.yticks([])
#     plt.xlim(-1, max(path_len)+1)

    cell = 0
    for i in range(len(cell_progression)):
        for j in range(len(cell_progression[i])):
            plt.plot([j, j+1], [cell, cell], color = myColors[cell_progression[i][j]], linewidth=line_width)
        cell = cell + 1
    #plt.ylabel("Cells")
    plt.xlabel("Trajectory")
    plt.yticks([])
    plt.xlim(-1, max(path_len)+1)

    cell_range = paths[0][paths[0].shape[0]-1,0] - paths[0][0,0]
    interval = cell_range/4
    cells3 = np.where((paths[0][:,0] > intermediate_coord[0]-interval)&(paths[0][:,0] < intermediate_coord[0]+interval))[0]
    
    #bounds = [0, np.min(cells3), np.max(cells3), paths[0].shape[0]-1]
    bounds = [0,20, 40, max(path_len)-1]
    #print(bounds)
    #norm = mpl.colors.Normalize(vmin=0,vmax=2)

#     n_clusters = 3
#     cmap = plt.get_cmap('Set3', n_clusters)
#     norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
#     sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
#     #sm.set_array([])

#     cbar = plt.colorbar(sm,
#                  orientation="horizontal")

#     #tick_locs = (np.arange(3) + 0.5)*(3-1)/3
#     tick_locs = [(0+bounds[1]/2, (bounds[1]+bounds[2]/2), (bounds[2]+bounds[3])/2)]
#     cbar.set_ticks(tick_locs)
#     cbar.set_ticklabels(['E','I','M'])

    fig, ax = plt.subplots(figsize=(6, 1))
    fig.subplots_adjust(bottom=0.6)
    
    cmap = plt.get_cmap('Set3', 3)

    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    #position=fig1.add_axes([0.93,0.1,0.02,0.35]) 
    
    cbar = fig.colorbar(
        mpl.cm.ScalarMappable(cmap=cmap, norm=norm),
        cax=ax,
        #cax=[0.93,0.1,0.02,0.35],
        spacing='proportional',
        orientation='horizontal')
    
    tick_locs = [(0+bounds[1]/2, (bounds[1]+bounds[2])/2, (bounds[2]+bounds[3])/2)]
    cbar.set_ticks(tick_locs)
    cbar.set_ticklabels(['Epithelial','Intermediate','Mesenchymal'])

    
    
    plt.show()

In [139]:
def plot_cell_progression_lineplot2(cell_progression, paths, intermediate_coord, line_width=10):
    interval_cell_progression = []
    cell_progression = np.array(cell_progression)
    path_len = []
    intermediate_len = []
    index = 0
    
    n_clusters = 5
    #cmap = plt.get_cmap('Plasma', n_clusters)
    cmap = plt.cm.get_cmap('Purples')
    
    #myColors = ((248/256,177/256,149/256),(246/256,114/256,128/256),(192/256,108/256,132/256),(108/256,91/256,123/256),(53/256,92/256,125/256))
    myColors = ((255/256,91/256,165/256),(219/256,73/256,172/256), (176/256,124/256,218/256), (107/256,167/256,214/256),(100/256,212/256,199/256))
    
    for i in range(paths.shape[0]):
        cell_range = paths[i][paths[i].shape[0]-1,0] - paths[i][0,0]
        interval = cell_range/3
        cells = np.where((paths[i][:,0] > intermediate_coord[0]-interval)&(paths[i][:,0] < intermediate_coord[0]+interval))[0]
        intermediate_len.append(cells.shape[0])
        path_len.append(paths[i].shape[0])
        
    min_path = min(path_len)
    min_intermediate = min(intermediate_len)
    print("Min length: ", min_path)
    print("Min intermediate: ", min_intermediate)
        
    for i in range(paths.shape[0]):    
        cell_range = paths[i][paths[i].shape[0]-1,0] - paths[i][0,0]
        interval = cell_range/3

        cells = np.where(paths[i][:,0] < intermediate_coord[0]-interval)[0]
        cells2 = np.where(paths[i][:,0] > intermediate_coord[0]+interval)[0]
        cells3 = np.where((paths[i][:,0] > intermediate_coord[0]-interval)&(paths[i][:,0] < intermediate_coord[0]+interval))[0]
        #outside_cells = np.concatenate((cells, cells2))
        
        print("start:", cells.shape[0])
        print("end:", cells2.shape[0])
        print("middle:", cells3.shape[0])
        
        #if cells3.shape[0] > min_intermediate:
        #    cells3 = random.sample(cells3.tolist(), min_intermediate)
        #    cells3 = np.array(cells3)
        
        for j in range(cells.shape[0]):
            #plt.plot([j-cells.shape[0], j+1-cells.shape[0]], [index, index], color = myColors[cell_progression[i][cells[j]]], linewidth=line_width )
            plt.plot([j-cells.shape[0], j+1-cells.shape[0]], [index, index], color = cmap(cell_progression[i][cells[j]]/phases.shape[0]), linewidth=line_width )
        
        for j in range(cells3.shape[0]):
            #plt.plot([j, j+1], [index, index], color = myColors[cell_progression[i][cells3[j]]], linewidth=line_width )
            plt.plot([j, j+1], [index, index], color = cmap(cell_progression[i][cells3[j]]/phases.shape[0]), linewidth=line_width )
            
        for j in range(cells2.shape[0]):
            #plt.plot([j+cells3.shape[0], j+1+cells3.shape[0]], [index, index], color = myColors[cell_progression[i][cells2[j]]], linewidth=line_width )
            plt.plot([j+cells3.shape[0], j+1+cells3.shape[0]], [index, index], color = cmap(cell_progression[i][cells2[j]]/phases.shape[0]), linewidth=line_width )
            
        index = index + 1
        
    plt.vlines(0,ymin=0, ymax=paths.shape[0]-0.5, linestyles='dashed')
    plt.vlines(min_intermediate,ymin=0, ymax=paths.shape[0]-0.5, linestyles='dashed')
        
#         outside_num = min_path - cells3.shape[0]
        
#         num_to_choose = int(outside_num/2)
#         beginning = []
#         ending = []
        
#         if cells.shape[0] < num_to_choose:
#             beginning = cells.tolist()
#             diff = num_to_choose - len(beginning)
#             ending = random.sample(cells2.tolist(), num_to_choose + diff)
            
#         if cells2.shape[0] < num_to_choose:
#             ending = cells2.tolist()
#             diff = num_to_choose - len(ending)
#             beginning = random.sample(cells.tolist(), num_to_choose + diff) 
            
#         if cells.shape[0] > num_to_choose & cells2.shape[0] > num_to_choose:
#             beginning = random.sample(cells.tolist(), num_to_choose) 
#             ending = random.sample(cells2.tolist(), num_to_choose)
        
#         if 0 not in beginning:
#             beginning.insert(0, 0)
#         if min_path not in ending:
#             ending.append(min_path)
#         indices = beginning + ending + cells3.tolist()

        

    # Plotting using multiple line plots
    import matplotlib.patches as mpatches

    #myColors = ((248/256,177/256,149/256),(246/256,114/256,128/256),(192/256,108/256,132/256),(108/256,91/256,123/256),(53/256,92/256,125/256))
    
    first_patch = mpatches.Patch(color=cmap(0/5), label=phases[0])
    sec_patch = mpatches.Patch(color=cmap(1/5), label=phases[1])
    third_patch = mpatches.Patch(color=cmap(2/5), label=phases[2])
    fourth_patch = mpatches.Patch(color=cmap(3/5), label=phases[3])
    fifth_patch = mpatches.Patch(color=cmap(4/5), label=phases[4])
    plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))

    
#     first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
#     sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
#     third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
#     fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
#     fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
#     plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))

    ax = plt.axes()
    ax.set_facecolor("white")

#     cell = 0
#     for i in range(len(interval_cell_progression)):
#         for j in range(len(interval_cell_progression[i])):
#             plt.plot([j, j+1], [cell, cell], color = myColors[interval_cell_progression[i][j]], linewidth=line_width)
#         cell = cell + 1
#     #plt.ylabel("Cells")
#     plt.xlabel("Trajectory")
#     plt.yticks([])
#     plt.xlim(-1, max(path_len)+1)

#     cell = 0
#     for i in range(len(cell_progression)):
#         for j in range(len(cell_progression[i])):
#             plt.plot([j, j+1], [cell, cell], color = myColors[cell_progression[i][j]], linewidth=line_width)
#         cell = cell + 1
    #plt.ylabel("Cells")
    plt.xlabel("Trajectory")
    plt.yticks([])
    #plt.xlim(-1, max(path_len)+1)

#     cell_range = paths[0][paths[0].shape[0]-1,0] - paths[0][0,0]
#     interval = cell_range/4
#     cells3 = np.where((paths[0][:,0] > intermediate_coord[0]-interval)&(paths[0][:,0] < intermediate_coord[0]+interval))[0]
    
    #bounds = [0, np.min(cells3), np.max(cells3), paths[0].shape[0]-1]
    #bounds = [0,20, 40, max(path_len)-1]
    #bounds = [-10, 0, min_intermediate, 20]
    #print(bounds)

#     n_clusters = 3
#     cmap = plt.get_cmap('Set3', n_clusters)
#     norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
#     sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
#     #sm.set_array([])
#     cbar = plt.colorbar(sm,
#                  orientation="horizontal")
#     #tick_locs = (np.arange(3) + 0.5)*(3-1)/3
#     tick_locs = [(0+bounds[1]/2, (bounds[1]+bounds[2]/2), (bounds[2]+bounds[3])/2)]
#     cbar.set_ticks(tick_locs)
#     cbar.set_ticklabels(['E','I','M'])

#     fig, ax = plt.subplots(figsize=(6, 1))
#     fig.subplots_adjust(bottom=0.6)
    
#     cmap = plt.get_cmap('Set3', 3)
#     norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

#     cbar = fig.colorbar(
#         mpl.cm.ScalarMappable(cmap=cmap, norm=norm),
#         cax=ax,
#         spacing='proportional',
#         orientation='horizontal')
    
#     tick_locs = [(0+bounds[1]/2, (bounds[1]+bounds[2])/2, (bounds[2]+bounds[3])/2)]
#     cbar.set_ticklabels(['Epithelial','Intermediate','Mesenchymal'])
    
    plt.show()

In [159]:
def plot_cell_progression_lineplot3(cell_progression, paths, mid, line_width=10):
    
    phases = np.array(['M-G1','G1-S', 'S','G2-M','M'], dtype=object)
    myColors = ((255/256,91/256,165/256),(219/256,73/256,172/256), (176/256,124/256,218/256), (107/256,167/256,214/256),(100/256,212/256,199/256))
    
    for i in range(paths.shape[0]):
        coords = np.vstack((paths[i][:,0],np.repeat(0,paths[i].shape[0]))).T
        index = getNNeighbors([mid, 0], coords, 1)[0][0]
        #print(paths[i].shape[0], index)
        x_val = -index
        #print(len(cell_progression[i]))
        #print(paths[i].shape[0])
        for j in range(paths[i].shape[0]-1):
            #print(cell_progression[i][j])
            #print(x_val, cell_progression[i][j])
            #print(paths[i].shape[0], j)
            plt.plot([x_val, x_val + 1], [i,i], color=myColors[cell_progression[i][j]], linewidth=line_width)
            x_val = x_val + 1
    first_patch = mpatches.Patch(color=myColors[0], label=phases[0])
    sec_patch = mpatches.Patch(color=myColors[1], label=phases[1])
    third_patch = mpatches.Patch(color=myColors[2], label=phases[2])
    fourth_patch = mpatches.Patch(color=myColors[3], label=phases[3])
    fifth_patch = mpatches.Patch(color=myColors[4], label=phases[4])
    plt.legend(handles=[first_patch, sec_patch, third_patch, fourth_patch, fifth_patch], loc = "upper left", bbox_to_anchor=(1, 1))
    plt.ylabel("Cells")
    plt.xlabel("Trajectory")
    plt.yticks([])
    plt.show()

In [156]:
def plot_heatmap(cell_progression, paths, intermediate_coord):
    lengths = []
    phases = np.array(['M-G1','G1-S', 'S','G2-M','M'], dtype=object)
    for i in range(cell_progression.shape[0]):
        lengths.append(len(cell_progression[i]))
    min_length = min(lengths)    
    
    #print(lengths)
    #middle_coord = []
    
    cell_prog = []
    cell_progression2 = cell_progression
    for i in range(cell_progression2.shape[0]):
        index = getNNeighbors(intermediate_coord, paths[i], 1)[0][0]
        #middle_coord.append(index)
        
        #print(len(cell_progression2[i]) - min_length)
        
        temp = cell_progression2[i]
        for j in range(len(cell_progression2[i]) - min_length):
            num = random.randint(0, len(temp)-1)
            temp.pop(num)
        cell_prog.append(temp)
        #print(temp, "\n")
        
    #print(middle_coord)
    #print(np.average(np.array(middle_coord)))
    
    cmap = sns.color_palette("rocket", phases.shape[0])
    ax = sns.heatmap(cell_prog, cmap=cmap)
    colorbar = ax.collections[0].colorbar
    r = colorbar.vmax - colorbar.vmin
    colorbar.set_ticks([colorbar.vmin + 0.5 * r / (phases.shape[0]) + r * i / (phases.shape[0]) for i in range(phases.shape[0])])
    colorbar.set_ticklabels(phases)
    ax.set_ylabel('Cells')
    ax.set_xlabel('Trajectory')
    ax.set(yticklabels=[])
    ax.set(xticklabels=[])

In [153]:
def get_ge(adata, path, gene_name):
    ge = []
    gene_index = np.where(adata.var['var_names'] == gene_name)[0][0]
    #print(path.shape[0])
    for i in range(path.shape[0]):
        indices = getNNeighbors(path[i], adata.obsm['X_umap'], 20)
        temp = adata.X[indices, gene_index].toarray()
        ge.append(np.average(temp))
    return ge

In [ ]:
def get_ge_paths(adata, paths, gene_name):
    ge = []
    for i in range(paths.shape[0]):
        ge.append(get_ge(adata, paths[i], gene_name))
    return ge

In [155]:
def plot_heatmap_ge(ge, paths, gene):
    lengths = []
    for i in range(paths.shape[0]):
        lengths.append(len(paths[i]))
    min_length = min(lengths)    

    final_ge = []
    for i in range(paths.shape[0]):
        temp = ge[i]
        for j in range(len(ge[i]) - min_length):
            num = random.randint(0, len(temp)-1)
            temp.pop(num)
        final_ge.append(temp)
        
    ax = sns.heatmap(final_ge, cmap=sns.color_palette("rocket", as_cmap=True))
    ax.set_ylabel('Cells')
    ax.set_xlabel('Trajectory')
    ax.set(yticklabels=[])
    #ax.set(xticklabels=[])
    ax.set_title(gene + " trajectory expression")
    plt.show()

In [26]:
def plot_traj_states(paths, adata):
    #Plotting trajectories into 3 states (epithelial, intermediate, mesenchymal)
    from matplotlib import cm
    n_clusters = 3
    #cmap = plt.get_cmap('Set3', n_clusters)
    cmap = colors.ListedColormap([(0, 85/256, 129/256), (114/256,205/256,244/256), (255/256, 210/256, 0)])
    states = ['Epithelial', 'Intermediate','Mesenchymal']

    for j in range(paths.shape[0]):
        final_coords = []

        x_coords = np.linspace(paths[j][0,0], paths[j][paths[j].shape[0]-1,0], 30)
        coords = np.vstack((paths[j][:,0], np.repeat(0, paths[j].shape[0]))).T
        interval_coords = np.vstack((x_coords, np.repeat(0, x_coords.shape[0]))).T

        plt.scatter(adata.obsm["X_umap"][:,0], adata.obsm["X_umap"][:,1], color = "lightgrey")

        for i in range(interval_coords.shape[0]): 
            index = getNNeighbors(interval_coords[i], coords, 1)

            final_coords.append(paths[j][index,:][0][0])
        final_coords = np.array(final_coords)
        interval = np.linspace(final_coords[0,0], final_coords[len(final_coords)-1,0], 4)

        ax = plt.axes()
        ax.set_facecolor("white")

        for i in range(1,len(interval)):
            indices = np.where((final_coords[:,0] < interval[i]) & (final_coords[:,0] >= interval[i-1]))[0]
            plt.scatter(final_coords[indices,0], final_coords[indices,1], color=cmap(i-1), label = states[i-1])
        plt.scatter(final_coords[0,0], final_coords[0,1], marker = "*", color = "red", label="start")
        plt.scatter(final_coords[final_coords.shape[0]-1,0], final_coords[final_coords.shape[0]-1,1], marker = "*", color = "black", label = "end")
        plt.legend(loc = "upper left", bbox_to_anchor=(1, 1))
        plt.axis('off')
        plt.show()

In [39]:
# Splits the trajectories into even thirds and plots the corresponding state colors 
def plot_traj_states2(paths, adata, intermediate_coord):
    #Plotting trajectories into 3 states (epithelial, intermediate, mesenchymal)
    from matplotlib import cm
    n_clusters = 3
    #cmap = plt.get_cmap('Set3', n_clusters)
    cmap = colors.ListedColormap([(0, 85/256, 129/256), (114/256,205/256,244/256), (255/256, 210/256, 0)])
    states = ['Epithelial', 'Intermediate','Mesenchymal']

    for i in range(paths.shape[0]):
            
        #state_coords = [paths[i][0][0], intermediate_coord, paths[i][paths[i].shape[0]-1]]    

        cells = int(paths[i].shape[0]/3)
        indices = getNNeighbors(intermediate_coord, paths[i], k=cells)[0]

        min_indices = np.min(indices)
        max_indices = np.max(indices)

        print(min_indices, max_indices, paths[i].shape[0])
        
        plt.scatter(adata.obsm["X_umap"][:,0], adata.obsm["X_umap"][:,1], color = "lightgrey")

        plt.scatter(paths[i][0:min_indices,0], paths[i][0:min_indices,1], color = cmap(0))
        plt.scatter(paths[i][min_indices:max_indices,0], paths[i][min_indices:max_indices,1], color = cmap(1))
        plt.scatter(paths[i][max_indices:paths[i].shape[0]-1,0], paths[i][max_indices:paths[i].shape[0]-1,1], color = cmap(2))

        plt.scatter(intermediate_coord[0], intermediate_coord[1], marker = '*', color = 'blue', label = 'intermediate')
        plt.scatter(paths[i][0,0], paths[i][0,1], marker = "*", color = "red", label="start")
        plt.scatter(paths[i][paths[i].shape[0]-1,0], paths[i][paths[i].shape[0]-1,1], marker = "*", color = "black", label = "end")
        plt.legend(loc = "upper left", bbox_to_anchor=(1, 1))
        plt.axis('off')
        plt.show()

In [121]:
# Splits the trajectories into thirds for each state
# Arbitrarily assigned a range for the intermediate state 
# All the other states considered the epithelial or mesenchymal state

def plot_traj_states3(paths, adata, intermediate_coord):
    #Plotting trajectories into 3 states (epithelial, intermediate, mesenchymal)
    from matplotlib import cm
    n_clusters = 3
    #cmap = plt.get_cmap('Set3', n_clusters)
    cmap = colors.ListedColormap([(0, 85/256, 129/256), (114/256,205/256,244/256), (255/256, 210/256, 0)])
    states = ['Epithelial', 'Intermediate','Mesenchymal']

    for i in range(paths.shape[0]):
        
        cell_range = paths[i][paths[i].shape[0]-1,0] - paths[i][0,0]
        #print(cell_range)
        print(paths[i].shape[0])
        interval = cell_range/3
        #print(interval)
        #print(intermediate_coord[0]-interval, intermediate_coord[0]+interval)
        
        cells = np.where(paths[i][:,0] < intermediate_coord[0]-interval)[0]
        cells2 = np.where(paths[i][:,0] > intermediate_coord[0]+interval)[0]
        cells3 = np.where((paths[i][:,0] > intermediate_coord[0]-interval)&(paths[i][:,0] < intermediate_coord[0]+interval))[0]
        outside_cells = np.concatenate((cells, cells2))
        
        plt.scatter(adata.obsm["X_umap"][:,0], adata.obsm["X_umap"][:,1], color = "lightgrey")
        
        plt.scatter(paths[i][cells,0], paths[i][cells,1], color = cmap(0))
        plt.scatter(paths[i][cells2,0], paths[i][cells2,1], color = cmap(1))
        plt.scatter(paths[i][cells3,0], paths[i][cells3,1], color = cmap(2))
        
        for j in range(0,paths[i].shape[0],10):
            plt.scatter(paths[i][j,0], paths[i][j,1], color = 'black')
        
        plt.scatter(intermediate_coord[0], intermediate_coord[1], marker = '*', color = 'blue', label = 'intermediate')
        plt.scatter(paths[i][0,0], paths[i][0,1], marker = "*", color = "red", label="start")
        plt.scatter(paths[i][paths[i].shape[0]-1,0], paths[i][paths[i].shape[0]-1,1], marker = "*", color = "black", label = "end")
        plt.legend(loc = "upper left", bbox_to_anchor=(1, 1))
        plt.axis('off')
        plt.show()
        

In [46]:
def plot_changing_gene_exp(gene_names, gene_regulation, groups, filename, threshold = 2, plot_all = True):
    changing_genes = []
    index = 1

    timepoints = np.repeat([range(len(gene_regulation[0]))], len(gene_names), axis=0).tolist()
    
    for i in range(len(gene_regulation)):
        if plot_all == False:
            if np.unique(gene_regulation[i]).shape[0] > 1:
                if (np.max(gene_regulation[i]) - np.min(gene_regulation[i])) >= threshold:
                    plt.plot(timepoints[i], gene_regulation[i], marker = 'o')
                    plt.title(gene_names[i])
                    plt.ylim(-0.5, groups+0.5-1)
                    index = index + 1
                    changing_genes.append(gene_names[i])
                    plt.savefig("Plots/EMT_trajectory/" + filename + "/"+gene_names[i]+"_progression", dpi=300)
                    plt.show()
        else:
            plt.plot(timepoints[i], gene_regulation[i], marker = 'o')
            plt.title(gene_names[i])
            plt.ylim(-0.5, groups+0.5-1)
            index = index + 1
            changing_genes.append(gene_names[i])
            plt.savefig("Plots/EMT_trajectory/" + filename + "/"+gene_names[i]+"_progression", dpi=300)
            plt.show()

In [13]:
def get_cell_ge(path_sol, gene_index, adata):
    gene_expression = []
    #indices = getNNeighbors(path_sol[i], adata.obsm['X_umap'], k=100)[0]
    
    #for i in range(len(gene_names)):
        #temp = []
    #print(indexes)
    #print(gene_names)
    for j in range(0,len(path_sol),20):
        indices = getNNeighbors(path_sol[j], adata.obsm['X_umap'], k=100)[0]
        temp = []
        for k in range(len(indices)):
            temp.append(adata.X[indices[k], gene_index])
#         temp_mean = []
#         for i in range(0, len(temp), 10):
#             if i+10 > len(temp):
#                 temp_mean.append(mean(temp[i:]))
#             else:
#                 temp_mean.append(mean(temp[i:i+10]))
        gene_expression.append(temp)
    return gene_expression

In [14]:
def plot_ge(gene_name, gene_expression):
    x_axis = np.arange(0, len(gene_expression)*len(gene_expression[0]), 1)
    for i in range(len(gene_expression)):
        #print(len(x_axis[i*len(gene_expression[i]):(i+1)*len(gene_expression[i])]))
        #print(len(gene_expression[i]))
        plt.bar(x_axis[i*len(gene_expression[i]):(i+1)*len(gene_expression[i])], gene_expression[i], color=(0.1, 0.1, 0.1, 0.1),  edgecolor='blue')
        plt.title(gene_name + " gene expression")
    plt.show()

In [15]:
def plotting_ge_trajectory(path_sol, gene_index, gene_name, adata):
    gene_expression = []
    
    for j in range(0,len(path_sol)):
        indices = getNNeighbors(path_sol[j], adata.obsm['X_umap'], k=100)[0]
        temp = 0
        for k in range(len(indices)):
            temp = temp + adata.X[indices[k], gene_index]
        gene_expression.append(temp/100)
    x_axis = np.arange(0, len(gene_expression), 1)
    # Plotting bar plot
    plt.bar(x_axis, gene_expression, color=(0.1, 0.1, 0.1, 0.1),  edgecolor='blue')
    
    plt.title(gene_name)
    plt.ylim(0,np.max(adata.X.toarray()))
    plt.show()

    #return gene_expression

In [25]:
def plotting_ge_trajectory_scatter(path_sol, gene_index, gene_name, adata, cells):
    gene_expression = []
    
    for j in range(0,len(path_sol),3):
        indices = getNNeighbors(path_sol[j], adata.obsm['X_umap'], k=cells)[0]
        #print(np.repeat(j, cells).shape)
        #print(adata.X.toarray()[indices, gene_index].shape)
        plt.scatter(np.repeat(j, cells), adata.X.toarray()[indices, gene_index])
        
    #x_axis = np.arange(0, len(gene_expression), 1)
    # Plotting bar plot
    #plt.bar(x_axis, gene_expression, color=(0.1, 0.1, 0.1, 0.1),  edgecolor='blue')
    
    plt.title(gene_name)
    #plt.ylim(0,np.max(adata.X.toarray()))
    plt.show()

    #return gene_expression

In [160]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [177]:
def louvain_clustering(adata, res):
    dose = np.unique(adata.obs['dosage'])[0]
    print(dose, ":")
    sc.tl.louvain(adata, resolution=res, key_added='louvain_r'+str(res), random_state=10)
    print(adata.obs['louvain_r'+str(res)].value_counts())
    print("\n")
    sc.pl.umap(adata, color=['louvain_r'+str(res)], palette=sc.pl.palettes.vega_20)
    sc.tl.rank_genes_groups(adata, groupby='louvain_r'+str(res), key_added='rank_genes_r' + str(res), groups=[0,1])
    sc.pl.rank_genes_groups(adata, key='rank_genes_r' + str(res), groups=['0'], fontsize=12)
    sc.pl.rank_genes_groups(adata, key='rank_genes_r' + str(res), groups=['1'], fontsize=12)
    return adata

In [3]:
def plotNormalizedCounts(data_df, gene):
    rc_unique = data_df['RC nneighbor'].unique()[np.argsort(data_df['RC nneighbor'].unique())]
    rc_mean_exp = []
    for i in rc_unique:
        temp = data_df[data_df['RC nneighbor']==i]
        mean_num = np.mean(temp[gene])
        rc_mean_exp.append(mean_num)
    rc_mean_exp = pd.Series(rc_mean_exp).fillna(0).tolist()
    #z = np.polyfit(rc_unique, rc_mean_exp, 8)
    #p = np.poly1d(z)
    #xp = np.linspace(1, 109, 104)

    fig,ax=plt.subplots(figsize=(7.5,5), dpi=300)
    plt.scatter(data_df['RC nneighbor'], data_df[gene], c=data_df['CC'].map(colors))
    plt.hlines(0, xmin=-1, xmax=115, linestyles='dashed')
    #plt.plot(rc_unique, p(xp), color='black')
    plt.xlabel('Reaction coordinate',fontsize=12,fontweight='bold')
    plt.ylabel('Normalized counts', fontsize=12,fontweight='bold')
    plt.xticks(fontsize=10,fontweight='bold')
    plt.yticks(fontsize=10,fontweight='bold')
    plt.margins(0.05)
    plt.title(gene+' expression along reaction coordinate', fontsize=14,fontweight='bold')
    plt.savefig("result/RC_exp/normalized/"+gene+".png", transparent=True, dpi=300)

In [ ]:
def plotOrigCounts(data_df, gene):
    rc_unique = data_df['RC nneighbor'].unique()[np.argsort(data_df['RC nneighbor'].unique())]
    rc_mean_exp = []
    for i in rc_unique:
        temp = data_df[data_df['RC nneighbor']==i]
        mean_num = np.mean(temp[gene])
        rc_mean_exp.append(mean_num)
    rc_mean_exp = pd.Series(rc_mean_exp).fillna(0).tolist()
    z = np.polyfit(rc_unique, rc_mean_exp, 8)
    p = np.poly1d(z)
    xp = np.linspace(1, 109, 104)

    fig,ax=plt.subplots(figsize=(7.5,5), dpi=300)
    plt.scatter(data_df['RC nneighbor'], data_df[gene], c=data_df['CC'].map(colors))
    #plt.hlines(0, xmin=-1, xmax=115, linestyles='dashed')
    plt.plot(rc_unique, p(xp), color='black')
    plt.xlabel('Reaction coordinate',fontsize=12,fontweight='bold')
    plt.ylabel('Original counts', fontsize=12,fontweight='bold')
    plt.xticks(fontsize=10,fontweight='bold')
    plt.yticks(fontsize=10,fontweight='bold')
    plt.margins(0.05)
    plt.title(gene+' expression along reaction coordinate', fontsize=14,fontweight='bold')
    plt.savefig("result/RC_exp/"+gene+".png", transparent=True, dpi=300)

## DTW

In [161]:
# Performs DTW using the first path in the paths array as the reference
# Returns the DTW distance and the warp_paths in a list
def align_DTW(paths, ref_path):
    warp_paths = []
    distances = []

    for i in range(0, paths.shape[0]):
        x = ref_path[:,1]
        y = paths[i][:,1]
        dtw_distance, warp_path = fastdtw(x, y, dist=euclidean)
        warp_paths.append(warp_path)
        distances.append(dtw_distance)
    return distances, warp_paths

In [163]:
# Plots the new trajectories 
def plot_warped_paths(paths, warp_paths, ref_path, adata):
    toReturn = []
    for i in range(len(warp_paths)):
        new_path = []
        index = 0
        while index < len(warp_paths[i])-1:
            x1, y1 = warp_paths[i][index]
            #total = paths[i+1][y1,1]
            total = paths[i][y1,1]
            num = 1
            cont_bol = True
            index += 1
            
            while cont_bol & (index <= len(warp_paths[i])-1):
                x2, y2 = warp_paths[i][index]
                if x1 == x2:
                    #total += paths[i+1][y2,1]
                    total += paths[i][y2,1]
                    index = index + 1
                    num+=1
                else:
                    cont_bol = False
            new_path.append(total/num)
            
        #if len(new_path)!= paths[0].shape[0]:
        if len(new_path)!= ref_path.shape[0]:
            x, y = warp_paths[i][len(warp_paths[i])-1]
            #print(new_path.shape)
            #new_path.append(paths[i+1][y,1])
            new_path.append(paths[i][y,1])
        print(i)
        plt.scatter(adata.obsm["X_umap"][:,0], adata.obsm["X_umap"][:,1], color = "lightgrey")
        #plt.scatter(paths[0][:,0], new_path)
        plt.scatter(ref_path[:,0], new_path)
        #plt.scatter(paths[i+1][:,0], paths[i+1][:,1], c=paths[i+1][:,0])
        plt.scatter(paths[i][:,0], paths[i][:,1], c=paths[i][:,0])
        plt.show()
        
        toReturn.append(np.vstack((ref_path[:,0], new_path)).T)
        
    return np.array(toReturn)

In [ ]:
# Plots the new trajectories 
def plot_warped_paths2(paths, warp_paths, ref_path, adata):
    toReturn = []
    for i in range(len(warp_paths)):
        new_path = []
        index = 0
        while index < len(warp_paths[i])-1:
            x1, y1 = warp_paths[i][index]
            #total = paths[i+1][y1,1]
            total = paths[i][y1,1]
            num = 1
            cont_bol = True
            index += 1
            
            while cont_bol & (index <= len(warp_paths[i])-1):
                x2, y2 = warp_paths[i][index]
                if x1 == x2:
                    #total += paths[i+1][y2,1]
                    total += paths[i][y2,1]
                    index = index + 1
                    num+=1
                else:
                    cont_bol = False
            new_path.append(total/num)
            
        #if len(new_path)!= paths[0].shape[0]:
        if len(new_path)!= ref_path.shape[0]:
            x, y = warp_paths[i][len(warp_paths[i])-1]
            #print(new_path.shape)
            #new_path.append(paths[i+1][y,1])
            new_path.append(paths[i][y,1])
        # print(i)
        # plt.scatter(adata.obsm["X_umap"][:,0], adata.obsm["X_umap"][:,1], color = "lightgrey")
        # #plt.scatter(paths[0][:,0], new_path)
        # plt.scatter(ref_path[:,0], new_path)
        # #plt.scatter(paths[i+1][:,0], paths[i+1][:,1], c=paths[i+1][:,0])
        # plt.scatter(paths[i][:,0], paths[i][:,1], c=paths[i][:,0])
        # plt.show()
        
        toReturn.append(np.vstack((ref_path[:,0], new_path)).T)
        
    return np.array(toReturn)

: 

In [167]:
# Calculates pairwise distances between list of trajectories 
def pairwise_DTW(paths):
    distances = []
    warp_paths = []
    for i in range(paths.shape[0]):
        dist_temp = []
        warp_temp = []
        for j in range(paths.shape[0]):
            x = paths[i][:,1]
            y = paths[j][:,1]
            dtw_distance, warp_path = fastdtw(x, y, dist=euclidean)
            dist_temp.append(dtw_distance)
            warp_temp.append(warp_path)
        distances.append(dist_temp)
        warp_paths.append(warp_temp)
    return distances, warp_paths

In [ ]:
# Returns the index of the trajectories that are the closest in distance
def min_dist(distances):
    min_list = []
    for i in range(len(distances)):
        temp = np.array(distances[i])
        order = np.argsort(temp)
        min_list.append(temp[order][1])
    traj1_index = min_list.index(min(min_list))
    #traj2_index = distances[traj1_index].index(min(min_list))
    traj2_index = np.where(np.array(distances[traj1_index]==min(min_list)))[0]
    if len(traj2_index) > 1:
        traj2_index = traj2_index[1]
    else:
        traj2_index = traj2_index[0]
    return traj1_index, traj2_index

In [ ]:
# Calculates the average of 2 trajectories 
def calc_avg_traj(path1, path2):
    toReturn = []
    for i in range(path2.shape[0]):
        toReturn.append((path1[i,1] + path2[i,1])/2)
    toReturn = np.vstack((path1[:,0], np.array(toReturn)))
    return toReturn

In [ ]:
# Obtains the new warped path for a single trajectory
def get_warped_path(paths, warp_path):
    new_path = []
    index = 0
    while index < len(warp_path)-1:
        x1, y1 = warp_path[index]
        total = paths[1][y1,1] # get traj y value
        num = 1
        cont_bol = True
        index += 1

        while cont_bol & (index <= len(warp_path)-1):
            x2, y2 = warp_path[index]
            # when multiple points are mapped to the same point
            if x1 == x2:
                total += paths[1][y2,1]
                index = index + 1
                num+=1
            else:
                cont_bol = False
        new_path.append(total/num)

    if len(new_path)!= paths[0].shape[0]:
        x, y = warp_path[len(warp_path)-1]
        new_path.append(paths[1][y,1])
        
    return np.array((paths[0][:,0], new_path)).T

In [ ]:
# Obtains the new warped path for a single trajectory
# paths
# warp_path
# expr : num time points x num of genes
def get_warped_expr(paths, warp_path, expr):
    new_path = []
    new_expr = []
    index = 0
    while index < len(warp_path)-1:
        x1, y1 = warp_path[index]
        #print('y1:', y1)
        total = paths[1][y1,1] # get traj y value
        curr_expr = expr[y1,:]
        num = 1
        cont_bol = True
        index += 1

        while cont_bol & (index <= len(warp_path)-1):
            x2, y2 = warp_path[index]
            #print('\ty2:', y2)
            # when multiple points are mapped to the same point
            if x1 == x2:
                total += paths[1][y2,1]
                curr_expr += expr[y2, :]
                index = index + 1
                num+=1
            else:
                cont_bol = False
        new_path.append(total/num)
        new_expr.append(curr_expr/num)

    if len(new_path)!= paths[0].shape[0]:
        x, y = warp_path[len(warp_path)-1]
        new_path.append(paths[1][y,1])
        new_expr.append(expr[y,:])
        
    return np.array((paths[0][:,0], new_path)).T, np.array(new_expr)

In [170]:
#Works for trajectory numbeers with multiples of 4 or 8
def get_average_traj(paths):
    #Find trajectories with the smallet distances between them
    n = paths.shape[0]
    final_avg = None

    for i in range(int(np.log2(n)-1)):
        # Pairs every trajectory using minimum distance
        new_paths = []
        n = paths.shape[0]
        for j in range(int(n/2)):
            # Getting closest pair of trajectories
            dist_pairwise, warped_pairwise = pairwise_DTW(paths)
            
            traj1, traj2 = min_dist(dist_pairwise)
            
            # Gets new warped path to matched indices 
            _, warp_path = fastdtw(paths[traj1], paths[traj2], dist=euclidean)
            new_path = get_warped_path([paths[traj1], paths[traj2]], warp_path)
            
            # Calculates average of the two paths
            average_path = calc_avg_traj(paths[traj1], new_path)
            new_paths.append(average_path.T)

            if traj1 > traj2:
                #print("traj1:", traj1)
                #print("traj2:", traj2)
                del dist_pairwise[traj1]
                del dist_pairwise[traj2]
                paths = np.delete(paths, traj1, 0)
                paths = np.delete(paths, traj2, 0)
            else:
                #print("traj1:", traj1)
                #print("traj2:", traj2)
                del dist_pairwise[traj2]
                del dist_pairwise[traj1]
                paths = np.delete(paths, traj2, 0)
                paths = np.delete(paths, traj1, 0)

        paths = np.array(new_paths)

        if len(new_paths) == 2:
            _, warp_path = fastdtw(new_paths[0], new_paths[1], dist=euclidean)
            new_path = get_warped_path([new_paths[0], new_paths[1]], warp_path)
            final_avg = calc_avg_traj(new_paths[0], new_path)

    return final_avg.T

In [1]:
# Obtains the cell progression of the warped path
# path[0] = avg path
# path[1] = path being warped
def get_cell_prog_warped_path(cell_prog, paths, warp_path):
    phases = np.array(['M-G1','G1-S', 'S','G2-M','M'], dtype=object)
    cell_prog_path = []
    index = 0
    while index < len(warp_path)-1:
        x1, y1 = warp_path[index]
        tracker = np.repeat(0, phases.shape[0])
        tracker[cell_prog[y1]] = tracker[cell_prog[y1]] + 1
        num = 1
        cont_bol = True
        index += 1

        while cont_bol & (index <= len(warp_path)-1):
            x2, y2 = warp_path[index]
            if x1 == x2:
                tracker[cell_prog[y2]] = tracker[cell_prog[y2]] + 1
                index = index + 1
                num+=1
            else:
                cont_bol = False
        max_phase = np.where(tracker == np.max(tracker))[0][0]
        cell_prog_path.append(max_phase)

    if len(cell_prog_path)!= paths[0].shape[0]:
        x, y = warp_path[len(warp_path)-1]
        cell_prog_path.append(cell_prog[y])
        
    return np.array(cell_prog_path)

In [ ]:
def plot_cell_prog_DTW(cell_progressions, paths, avg_path, ):
    for i in range(paths.shape[0]):
        _, warped_path = fastdtw(avg_path, paths[i], dist=euclidean)
        

## Jacobian

In [3]:
# Plots the jacobian of each cell in umap space
def jacobian_cell_cell(adata, reg, eff):
    dyn.pl.jacobian(adata, regulators=[reg], effectors=[eff], basis='umap')
    dyn.pl.jacobian(adata, regulators=[eff], effectors=[reg], basis='umap')
    dyn.pl.jacobian(adata, regulators=[reg], effectors=[reg], basis='umap')
    dyn.pl.jacobian(adata, regulators=[eff], effectors=[eff], basis='umap')

In [4]:
# Plots the jacobian of each cell as a function of gene concentration
def jacobian_ge_level(adata, reg, eff):
    dyn.pl.jacobian(adata, regulators=[reg], effectors=[eff], x=reg, y=eff, layer='M_s', basis='umap')
    dyn.pl.jacobian(adata, regulators=[eff], effectors=[reg], x=eff, y=reg, layer='M_s', basis='umap')
    #dyn.pl.jacobian(adata, regulators=[reg], effectors=[reg], x=reg, y=eff, layer='M_s', basis='umap')
    #dyn.pl.jacobian(adata, regulators=[eff], effectors=[eff], x=eff, y=eff, layer='M_s', basis='umap')

In [6]:
#Looks at the top interactions
def plot_jacobian_TF_reg(adata, reg, top_genes, dose, genes=None):
    if genes == None:
        dyn.pp.top_pca_genes(adata, n_top_genes=100)
        top_pca_genes = adata.var.index[adata.var.top_pca_genes];
        genes = top_pca_genes.tolist()
        genes.append('SNAI1')
        genes.append('SNAI2')
        genes.append('TWIST1')
        genes.append('TWIST2')
        genes.append('ZEB1')
        genes.append('ZEB2')
    else:
        top_genes = len(genes)
    
    dyn.vf.jacobian(adata, regulators=[reg] , effectors=genes)
    dyn.vf.rank_jacobian_genes(adata,
                           mode="int",
                           abs=True,
                           output_values=True)
    
    for i in range(adata.uns['int'].shape[0]):
        eff, reg = np.array(adata.uns['int'])[i,0].split(" - ")
        file_name = "Plots/TF_regulation/Dose" + dose + "/" + reg + "/" + reg + "_" + eff + "_umap_"
        print(eff, reg)
        dyn.pl.jacobian(adata, regulators=[reg], effectors=[eff], basis='umap', save_show_or_return='save', save_kwargs = {"path":file_name, "ext":'png'})
        
    for i in range(adata.uns['int'].shape[0]):
        eff, reg = np.array(adata.uns['int'])[i,0].split(" - ")
        file_name = "Plots/TF_regulation/Dose" + dose + "/" + reg + "/" + reg + "_" + eff + "_xy_"
        print(eff, reg)
        dyn.pl.jacobian(adata, regulators=[reg], effectors=[eff], x=reg, y=eff, layer='M_s', basis='umap', save_show_or_return='save', save_kwargs = {"path":file_name, "ext":'png'})

In [16]:
def plot_gene_perturbation(adata, perturb_gene, eff, exp):
    dose = np.unique(adata.obs['dosage'])[0]
    dyn.pd.perturbation(adata, genes=[perturb_gene], expression = exp, perturb_mode="raw")
    dV = vector_transformation(adata.obsm['j_delta_x_perturbation'], adata.uns['PCs'])

    all_genes = adata.var_names[adata.var['use_for_pca']]
    gene_index = None
    if eff in all_genes:
        gene_index = all_genes.get_loc(eff)

    dv = dV[:,gene_index]
    adata.obs['dv_' + eff] = dv
    if exp < 0:
        file_name = "Plots/Dose12.5-regulation/"+perturb_gene+"/neg_pert_dv_" + eff 
    else:
        file_name = "Plots/Dose12.5-regulation/"+perturb_gene+"/pos_pert_dv_" + eff 
        
    #plt.scatter(adata.obsm['X_umap'][:,0], adata.obsm['X_umap'][:,1], c=adata.obs['dv_' + eff], alpha=0.5, cmap='viridis')
    plt.scatter(adata.obsm['X_umap'][:,0], adata.obsm['X_umap'][:,1], norm=DivergingNorm(0), c=adata.obs['dv_' + eff], alpha=0.5, cmap='bwr')
    plt.title("Dose " + dose + ": " + perturb_gene + " peturbation")
    plt.xlabel("dv_" + eff)
    plt.colorbar(orientation="vertical")
    plt.tight_layout()
    plt.savefig(file_name, dpi=300)
    plt.show()
    
    
    dyn.pl.streamline_plot(adata, basis='umap_perturbation', color='dv_' + eff)

## Scoring function

In [17]:
# Calculates the EMT score using correlation coefficients to e-cadherin 
# gene_exp = raw data, M_s, log transformed gene expression
# Adds EMT score and final EMT score to obs of the adata

def calc_EMT_score(adata, gene_coeff, gene_exp, threshold):
    EMT_score = []
    all_genes = adata.var_names
    
    for i in range(gene_exp.shape[0]):
        score = 0
        for j in range(gene_coeff.shape[0]):
            gene = gene_coeff['GENE'][j] 
            if gene in adata.var_names.tolist():
                gene_index = None
                if gene in all_genes:
                    gene_index = all_genes.get_loc(gene)
                    score += gene_coeff['E-cad'][j]*gene_exp[i,gene_index]
        EMT_score.append(score)
        
    adata.obs['EMT_score'] = EMT_score

    final_EMT_score = []
    for i in range(len(EMT_score)):
        if EMT_score[i] < threshold:
            final_EMT_score.append('M')
        else:
            final_EMT_score.append('E')
    adata.obs['final_EMT_score'] = final_EMT_score
    return adata

In [ ]:
# Calculates the EMT score using correlation coefficients to e-cadherin 
# gene_exp = raw data, M_s, log transformed gene expression
# Adds EMT score and final EMT score to obs of the adata

def calc_EMT_score2(adata, gene_coeff, gene_exp, threshold):
    EMT_score = []
    all_genes = adata.var_names
    
    for i in range(gene_exp.shape[0]):
        score = 0
        for j in range(gene_coeff.shape[0]):
            gene = gene_coeff['GENE'][j] 
            #print(gene)
            #if gene in adata.var_names.tolist():
            #    gene_index = None
            if gene in all_genes:
                gene_index = all_genes.get_loc(gene)
                score += gene_coeff['E-cad'][j]*gene_exp[gene][i]
        EMT_score.append(score)
        
    #adata.obs['EMT_score'] = EMT_score

    final_EMT_score = []
    for i in range(len(EMT_score)):
        if EMT_score[i] < threshold:
            final_EMT_score.append('M')
        else:
            final_EMT_score.append('E')
    #adata.obs['final_EMT_score'] = final_EMT_score
    return EMT_score, final_EMT_score

In [18]:
def normalize_expr(expr_dict):
    ret = {}
    for k, v in expr_dict.items():      
        if np.min(v)==0 and np.max(v)==0:
            ret[k] = v
        else:
            ret[k] = (v - np.mean(v)) / (np.max(v) - np.min(v))
    return ret

def emt_score(norm_expr_dict, k):
    ret = np.zeros(k)
    for i in range(k):
        ret[i] = norm_expr_dict['COL1A2'][i] + norm_expr_dict['FN1'][i] + norm_expr_dict['SNAI1'][i] \
            + norm_expr_dict['SNAI2'][i] \
            + norm_expr_dict['VIM'][i] + norm_expr_dict['ZEB1'][i] \
            - norm_expr_dict['ERBB3'][i] - norm_expr_dict['KRT15'][i] - norm_expr_dict['PSCA'][i] - norm_expr_dict['MAL2'][i] \
            - norm_expr_dict['S100P'][i]
    return ret

In [ ]:
def KS_emt_score(E_exp, M_exp):
    finalScore = []
    for i in range(E_exp.shape[0]):
        #stat, pval = ks_2samp(E_exp.iloc[i,:], M_exp.iloc[i,:])
        stat2, pval2 = ks_2samp(E_exp.iloc[i,:], M_exp.iloc[i,:], alternative='less')
        stat3, pval3 = ks_2samp(E_exp.iloc[i,:], M_exp.iloc[i,:], alternative='greater')

        if pval2 < 0.05:
            finalScore.append(-1*stat2)
        elif pval3 < 0.05:
            finalScore.append(stat3)
        else:
            if stat3 > stat2:
                finalScore.append(stat3)
            else:
                finalScore.append(-1*stat2)
    return finalScore

## Other

In [12]:
# converts pM to ng/ml
# 1 ng/ml = 40 pM
def convert_pM(nums):
    toReturn = []
    for i in range(len(nums)):
        toReturn.append(nums[i]/40.0)
    return toReturn

In [14]:
# converts ng/ml to pM
# 1 ng/ml = 40 pM
def convert_ngml(nums):
    toReturn = []
    for i in range(len(nums)):
        toReturn.append(nums[i]*40.0)
    return toReturn

In [3]:
# Prints whether or not gene is in given gene list
def findGene(gene_name, gene_list):
    inSet = gene_name in gene_list
    if inSet == True:
        print(gene_name, "\t True")
    else:
        print(gene_name, "\t False")
    return inSet

In [ ]:
def getLOG2_FoldChange(expr):
    toReturn = [0]
    control = expr[0]
    for i in range(1,len(expr)):
        toReturn.append(np.log2(expr[i]/control))
    return toReturn

In [3]:
def getFoldChange(expr):
    toReturn = [0]
    control = None
    if expr[0] == 0:
        control = 0.001
    else:
        control = expr[0]
    for i in range(1,len(expr)):
        toReturn.append((expr[i] - control)/control)
    return toReturn

In [ ]:
def plot_expression(gene, dose_cells, process_adata2, folder):
    intervals = np.linspace(-1,1,4)
    exp_mean = []
    exp_std = []
    exp = []
    my_cmap = plt.get_cmap("viridis")
    y = range(8)
    rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))
    plt.figure(figsize=(5,3.5), dpi=100)

    interval_top = None
    interval_bottom = None

    group_index = [0,1,1,1,2,2,2,2]
    for i in range(len(dose_cells)):
        if i == 0:
            interval_top = intervals[group_index[i]+2]
            interval_bottom = intervals[group_index[i]]
        else:
            interval_top = intervals[group_index[i]+1]
            interval_bottom = intervals[group_index[i]]

        adata_temp = processed_adata2[np.where(processed_adata2.obs['dosage'] == str(doses[i]))[0],:]
        indexes = np.where((adata_temp.obs['emt_score'] < interval_top) & (adata_temp.obs['emt_score'] >= interval_bottom))[0]
        gene_names = adata_temp.var.index
        gene_index = gene_names.get_loc(gene)
        expr = adata_temp.X.toarray()[indexes, gene_index]
        exp_mean.append(np.mean(expr))
        exp_std.append(np.std(expr))
        exp.append(expr)

    fc = getLOG2_FoldChange(exp_mean)

    plt.bar(range(8), exp_mean, color=my_cmap(rescale(y)))
    plt.xticks([0,1,2,3,4,5,6,7], doses)
    plt.xlabel("TGFB dose")
    plt.ylabel("Gene Expression")
    plt.title(gene + " expression across TGFB doses")
    plt.tight_layout()
    plt.savefig("Plots/TF_expression/" + folder +"/"+ gene + "_ge_3_selected_groups.png", dpi=200)
    plt.show()

    plt.figure(figsize=(5,3.5), dpi=100)
    plt.bar(range(7), fc, color=my_cmap(rescale(y)[1:]))
    plt.xticks([0,1,2,3,4,5,6], doses[1:])
    plt.xlabel("TGFB dose")
    plt.ylabel("log2 fold change")
    plt.title(gene + " expression across TGFB doses")
    plt.tight_layout()
    plt.savefig("Plots/TF_expression/"+folder+"/"+gene+"_fc_3_selected_groups.png", dpi=200)
    plt.show()

    plt.figure(figsize=(5,3.5), dpi=100)
    plt.bar(range(7), fc, color=my_cmap(rescale(y)[1:]))
    plt.xticks([0,1,2,3,4,5,6], converted_units[1:])
    plt.xlabel("TGFB dose")
    plt.ylabel("log2 fold change")
    plt.title(gene + " expression across TGFB doses")
    plt.tight_layout()
    plt.savefig("Plots/TF_expression/"+folder+"/"+gene+"_fc_3_selected_groups_JZ_units.png", dpi=200)
    plt.show()

In [ ]:
def plot_expression_A549(gene, adata_timepoints, folder):
    exp_mean = []
    exp = []
    my_cmap = plt.get_cmap("viridis")
    y = range(4)
    rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))
    plt.figure(figsize=(5,3.5), dpi=100)

    for i in range(len(timepoints)):
        adata_temp = adata_timepoints[np.where(adata_timepoints.obs['treatment'] == timepoints[i])[0],:]
        gene_names = adata_temp.var.index
        gene_index = gene_names.get_loc(gene)
        expr = adata_temp.X.toarray()[:, gene_index]
        exp_mean.append(np.mean(expr))
        exp.append(expr)

    fc = getLOG2_FoldChange(exp_mean)  

    plt.bar(range(4), exp_mean, color=my_cmap(rescale(y)))
    plt.xticks([0,1,2,3], timepoints)
    plt.xlabel("Timepoint")
    plt.ylabel("Gene Expression")
    plt.title(gene + "expression with time")
    plt.tight_layout()
    plt.savefig("../Plots/TF_expression/A549/"+folder+"/"+gene+"_ge.png", dpi=200)
    plt.show()

    plt.figure(figsize=(5,3.5), dpi=100)
    plt.bar(range(3), fc, color=my_cmap(rescale(y)[1:]))
    plt.xticks([0,1,2], timepoints[1:])
    plt.xlabel("Timepoint")
    plt.ylabel("log2 fold change")
    plt.title(gene + "expression with time")
    plt.tight_layout()
    plt.savefig("../Plots/TF_expression/A549/"+folder+"/"+gene+"_fc.png", dpi=200)
    plt.show()

In [ ]:
def drem2_normalize(expr):
    for i in range(expr.shape[0]):
        start = expr[i][0]
        expr[i][1:] = np.log2(expr[i][1:]/start)
    return expr

In [1]:
def gseapy_barplot(df, column='Adjusted P-value', title="", cutoff=0.05, top_term=10,
            figsize=(6.5,6), color='salmon', ofname=None, **kwargs):
    """Visualize enrichr results.
    :param df: GSEApy DataFrame results.
    :param column: which column of DataFrame to show. Default: Adjusted P-value
    :param title: figure title.
    :param cutoff: terms with 'column' value < cut-off are shown.
    :param top_term: number of top enriched terms to show.
    :param figsize: tuple, matplotlib figsize.
    :param color: color for bars.
    :param ofname: output file name. If None, don't save figure    
    
    """

    colname = column   
    if colname in ['Adjusted P-value', 'P-value']: 
        # check if any values in `df[colname]` can't be coerced to floats
        can_be_coerced = df[colname].map(isfloat)
        if np.sum(~can_be_coerced) > 0:
            raise ValueError('some value in %s could not be typecast to `float`'%colname)
        else:
            df.loc[:, colname] = df[colname].map(float)
        df = df[df[colname] <= cutoff]
        if len(df) < 1: 
            msg = "Warning: No enrich terms using library %s when cutoff = %s"%(title, cutoff)
            return msg
        df = df.assign(logAP = lambda x: - x[colname].apply(np.log10))
        colname = 'logAP' 
        
    dd = df.sort_values(by=colname).iloc[-top_term:,:]
    # create bar plot
    if hasattr(sys, 'ps1') and (ofname is None):
        # working inside python console, show (True) figure
        fig = plt.figure(figsize=figsize)
    else:
        # If working on commandline, don't show figure
        fig = Figure(figsize=figsize)
        canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    bar = dd.plot.barh(x='Term', y=colname, color=color, 
                       alpha=0.75, fontsize=9, ax=ax)
    
    if column in ['Adjusted P-value', 'P-value']:
        xlabel = "-log$_{10}$(%s)"%column
    else:
        xlabel = column 
    bar.set_xlabel(xlabel, fontsize=9, fontweight='bold')
    bar.set_ylabel("")
    bar.set_title(title, fontsize=12, fontweight='bold')
    bar.xaxis.set_major_locator(MaxNLocator(integer=True))
    bar.legend_.remove()
    adjust_spines(ax, spines=['left','bottom'])

    if hasattr(sys, 'ps1') and (ofname is not None): 
        # canvas.print_figure(ofname, bbox_inches='tight', dpi=300)
        fig.savefig(ofname, bbox_inches='tight', dpi=300)
        return
    elif ofname is None:
        return
    elif not hasattr(sys, 'ps1') and (ofname is not None):
        canvas.print_figure(ofname, bbox_inches='tight', dpi=300)
        return
    return ax

In [5]:
# Plots the gene expression for a given basis
def plotGeneExpression(adata, gene, basis):
    gene_index = adata.var.index.get_loc(gene)
    exp = adata.X.toarray()[:,gene_index]
    temp_indices = np.where(exp==0)
    plt.scatter(adata.obsm['X_'+basis][temp_indices,0], adata.obsm['X_'+basis][temp_indices,1], c='gainsboro', s=75)
    temp_indices = np.where(exp!=0)
#     print(adata.X.toarray()[temp_indices,gene_index])
#     print(adata.obsm['X_'+basis][temp_indices,0])
#     print(adata.obsm['X_'+basis][temp_indices,1])
    plt.scatter(adata.obsm['X_'+basis][temp_indices,0], adata.obsm['X_'+basis][temp_indices,1], c=adata.X.toarray()[temp_indices,gene_index][0], s=75, vmin=0)
    plt.title(gene)
    plt.colorbar()
    plt.show()

## Weikang's functions

In [2]:
#https://gist.github.com/pv/8036995
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi

def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

In [3]:
def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [ ]:
def get_directed_edges(trans_matrix):
    edges={}
    for i in range(trans_matrix.shape[0]):
        for j in range(trans_matrix.shape[1]):
            if i!=j and trans_matrix[i,j]>1e-2:
                edges[(i,j)]=-np.log(trans_matrix[i,j])
    return edges
def get_undirected_edges(trans_matrix):
        edges={}
        for i in range(trans_matrix.shape[0]):
            for j in range(i+1,trans_matrix.shape[1]):
                if trans_matrix[i,j]+trans_matrix[j,i]>2e-2:
                    edges[(i,j)]=trans_matrix[i,j]+trans_matrix[j,i]
        return edges

# def get_directed_edges(trans_matrix):
#     conn_indices = np.where(trans_matrix)
#     weights = trans_matrix[conn_indices]
#     edges = zip(*conn_indices)
#     edge_wts = dict(zip(edges, weights))
#     return edge_wts
 
def build_trans_networkx(trans_matrix,node_list,direct_flag=True):
    
    if direct_flag:
        G=nx.DiGraph()
        G.add_nodes_from(node_list)    
        edges_wts=get_directed_edges(trans_matrix)
    # edges represent transition probabilities
        for k, v in edges_wts.items():
            tmp_origin, tmp_destination = k[0], k[1]
            G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
    else:
        G=nx.Graph()
        G.add_nodes_from(node_list)    
        edges_wts=get_undirected_edges(trans_matrix)
        # edges represent transition probabilities
        for k, v in edges_wts.items():
            tmp_origin, tmp_destination = k[0], k[1]
            G.add_edge(tmp_origin, tmp_destination, weight=v, label=v)
        
    return G

In [11]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.interpolate import splprep,splev
from sklearn.cluster import DBSCAN
from scipy import interpolate
from numpy import linalg 
"""
Find path connecting data point
"""
class FindPath2(object):
    def __init__(self, n = 20,traj_w=10,sf=1, ):
        self.n = n
        self.t = np.linspace(0,1,n)
        self.error = 1e100
        self.error_log=[]
        self.data_rc=[]
        self.tangent_v=[]
        self.w=traj_w
        self.sf=sf
        #self.coords = coords
        #self.spline = spline
        #self.path = path
        
    def fit(self, p1, p2, data):
        self.p1 = np.expand_dims(p1,axis=0)
        self.p2 = np.expand_dims(p2,axis=0)
        self.data = data
        self.cluster_labels=[]

        self.spline = p1 + np.multiply((p2 - p1) , np.expand_dims(self.t,axis=1))
        #print(self.spline)
        # plt.scatter(self.coords['DC1'],
        #             self.coords['DC2'],
        #             c='gainsboro', zorder=1)
        plt.plot(self.spline[:,0], self.spline[:,1], zorder=2) 
        plt.show()
        
        self.best_spline=self.spline.copy()
        
        update=True
        while update:
            update=self.__move()
            print(update)
        return self.best_spline,self.data_rc
    
    def __move(self):
        w=self.w
        traj_error_arr1,traj_sum1,pts_error_arr1,pts_sum1,density1,data_rc1=self.__error() 
        centers = self.spline.copy()
#         print(pts_sum1) 
        
        updateIndexs = np.where(density1>0)[0]
        centers[updateIndexs,:]=(np.divide(pts_sum1[updateIndexs,:],np.expand_dims(density1[updateIndexs],axis=1))+\
                                w*traj_sum1[updateIndexs,:]/len(self.data))/(1+w)


        self.spline,tangent_v=self.__discretize(centers)
        
        dot_color=np.arange(self.spline.shape[0])
        cm=plt.cm.get_cmap('jet')
        plt.figure(figsize=(7.5,5))
        # plt.scatter(self.coords['DC1'],
        #             self.coords['DC2'],
        #             c='gainsboro', zorder=1)
        plt.plot(self.spline[:,0],self.spline[:,1], linestyle='dotted', c='black', zorder=2)
        plt.scatter(self.spline[:,0],self.spline[:,1],c=dot_color,cmap=cm, zorder=3)
        plt.xlabel('X',fontsize=16)
        plt.ylabel('Y',fontsize=16)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        #plt.savefig(self.path + "/" + str(self.error)+'.png',dpi=300)
        plt.show()
#         print(tangent_v)
        traj_error_arr2,traj_sum2,pts_error_arr2,pts_sum2,density2,data_rc2=self.__error() 

#         print(w*np.sum(traj_error_arr2),np.sum(pts_error_arr2))
        error=w*np.sum(traj_error_arr2)+np.sum(pts_error_arr2)
#         print(self.error_log)

        self.error_log.append(error)

        if error<self.error:
            self.error=error
            self.best_spline=self.spline.copy()
            self.data_rc=data_rc2
            self.tangent_v=tangent_v
            return True
        else:
            return False

    def __discretize(self,centers,sample_n=1,st_move=0.02):
        #centers[0,:] = self.p1
        #centers[-1,:] = self.p2
        spline=centers.copy()
        centers[0,:]=self.p1#+st_move*(centers[0,:]-self.p1)#self.spline[0,:]+st_move*(centers[0,:]-self.spline[0,:])
        centers[-1,:]=self.p2#+st_move*(centers[-1,:]-self.p2)#self.spline[-1,:]+st_move*(centers[-1,:]-self.spline[-1,:])
        t = np.linspace(0,1, sample_n* self.n)


        center_list=[centers[:,i] for i in range(centers.shape[1])]
        tck, u = interpolate.splprep(center_list,s=self.sf,k=2)
        spline_list=splev(t,tck,ext=3)
        for k in range(len(spline_list)):
            spline[:,k]=spline_list[k][::sample_n]
        
        
#         plt.title('smooth')
#         plt.plot(self.spline[:,0], self.spline[:,1]) 
#         plt.show()
#         plt.plot(self.spline[:,2], self.spline[:,3]) 
#         plt.show()
        
        tangent_v=interpolate.splev(t,tck,der=1)
        
        return spline,tangent_v
        
    def __error(self):
          
        pts_error_arr=np.zeros(self.n)
        density = np.zeros(self.n)
        pts_sum=np.zeros_like(self.spline)
        
        traj_error_arr=np.zeros(self.n)
        traj_sum=np.zeros_like(self.spline)
        data_rc=[]
        for ind in range(len(self.data)):
            traj_t_span = self.data[ind].shape[0]
            spline_inds=[]
            spline_dist=[]
            for j in range(self.n):
                p=self.spline[j]
                #print(self.data[ind])
                #print(p[None,:])
                dist=linalg.norm(self.data[ind]-p[None,:],axis=1)
#                 print(dist.shape)
                traj_error_arr[j]+=np.amin(dist)
                traj_sum[j]+=self.data[ind][np.argmin(dist),:]
                

            traj_rc=np.zeros((self.data[ind].shape[0],))
            for i in range(traj_t_span):
                p = self.data[ind][i,:]
                dist = linalg.norm(self.spline - p[None,:], axis = 1)
                pts_error_arr[np.argmin(dist)]+=np.amin(dist)
                pts_sum[np.argmin(dist)]+=self.data[ind][i,:]
                density[np.argmin(dist)]+=1
                traj_rc[i]=np.argmin(dist)
            data_rc.append(traj_rc)
        
        return traj_error_arr,traj_sum,pts_error_arr,pts_sum,density,data_rc

In [9]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.interpolate import splprep,splev
from sklearn.cluster import DBSCAN
from scipy import interpolate
from numpy import linalg 
"""
Find path connecting data point
"""
class FindPath(object):
    def __init__(self, coords, spline, path, n = 20,traj_w=10,sf=1, ):
        self.n = n
        self.t = np.linspace(0,1,n)
        self.error = 1e100
        self.error_log=[]
        self.data_rc=[]
        self.tangent_v=[]
        self.w=traj_w
        self.sf=sf
        self.coords = coords
        self.spline = spline
        self.path = path
        
    def fit(self, p1, p2, data):
        self.p1 = np.expand_dims(p1,axis=0)
        self.p2 = np.expand_dims(p2,axis=0)
        self.data = data
        self.cluster_labels=[]

        #self.spline = p1 + np.multiply((p2 - p1) , np.expand_dims(self.t,axis=1))
        plt.scatter(self.coords['DC1'],
                    self.coords['DC2'],
                    c='gainsboro', zorder=1)
        plt.plot(self.spline[:,0], self.spline[:,1], zorder=2) 
        plt.show()
        
        self.best_spline=self.spline.copy()
        
        update=True
        while update:
            update=self.__move()
            print(update)
        return self.best_spline,self.data_rc
    
    def __move(self):
        w=self.w
        traj_error_arr1,traj_sum1,pts_error_arr1,pts_sum1,density1,data_rc1=self.__error() 
        centers = self.spline.copy()
#         print(pts_sum1) 
        
        updateIndexs = np.where(density1>0)[0]
        centers[updateIndexs,:]=(np.divide(pts_sum1[updateIndexs,:],np.expand_dims(density1[updateIndexs],axis=1))+\
                                w*traj_sum1[updateIndexs,:]/len(self.data))/(1+w)


        self.spline,tangent_v=self.__discretize(centers)
        
        dot_color=np.arange(self.spline.shape[0])
        cm=plt.cm.get_cmap('jet')
        plt.figure(figsize=(7.5,5))
        plt.scatter(self.coords['DC1'],
                    self.coords['DC2'],
                    c='gainsboro', zorder=1)
        plt.plot(self.spline[:,0],self.spline[:,1], linestyle='dotted', c='black', zorder=2)
        plt.scatter(self.spline[:,0],self.spline[:,1],c=dot_color,cmap=cm, zorder=3)
        plt.xlabel('X',fontsize=16)
        plt.ylabel('Y',fontsize=16)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.savefig(self.path + "/" + str(self.error)+'.png',dpi=300)
        plt.show()
#         print(tangent_v)
        traj_error_arr2,traj_sum2,pts_error_arr2,pts_sum2,density2,data_rc2=self.__error() 

#         print(w*np.sum(traj_error_arr2),np.sum(pts_error_arr2))
        error=w*np.sum(traj_error_arr2)+np.sum(pts_error_arr2)
#         print(self.error_log)

        self.error_log.append(error)

        if error<self.error:
            self.error=error
            self.best_spline=self.spline.copy()
            self.data_rc=data_rc2
            self.tangent_v=tangent_v
            return True
        else:
            return False

    def __discretize(self,centers,sample_n=1,st_move=0.02):
        #centers[0,:] = self.p1
        #centers[-1,:] = self.p2
        spline=centers.copy()
        centers[0,:]=self.p1#+st_move*(centers[0,:]-self.p1)#self.spline[0,:]+st_move*(centers[0,:]-self.spline[0,:])
        centers[-1,:]=self.p2#+st_move*(centers[-1,:]-self.p2)#self.spline[-1,:]+st_move*(centers[-1,:]-self.spline[-1,:])
        t = np.linspace(0,1, sample_n* self.n)


        center_list=[centers[:,i] for i in range(centers.shape[1])]
        tck, u = interpolate.splprep(center_list,s=self.sf,k=2)
        spline_list=splev(t,tck,ext=3)
        for k in range(len(spline_list)):
            spline[:,k]=spline_list[k][::sample_n]
        
        
#         plt.title('smooth')
#         plt.plot(self.spline[:,0], self.spline[:,1]) 
#         plt.show()
#         plt.plot(self.spline[:,2], self.spline[:,3]) 
#         plt.show()
        
        tangent_v=interpolate.splev(t,tck,der=1)
        
        return spline,tangent_v
        
    def __error(self):
          
        pts_error_arr=np.zeros(self.n)
        density = np.zeros(self.n)
        pts_sum=np.zeros_like(self.spline)
        
        traj_error_arr=np.zeros(self.n)
        traj_sum=np.zeros_like(self.spline)
        data_rc=[]
        for ind in range(len(self.data)):
            traj_t_span = self.data[ind].shape[0]
            spline_inds=[]
            spline_dist=[]
            for j in range(self.n):
                p=self.spline[j]
                
                dist=linalg.norm(self.data[ind]-p[None,:],axis=1)
#                 print(dist.shape)
                traj_error_arr[j]+=np.amin(dist)
                traj_sum[j]+=self.data[ind][np.argmin(dist),:]
                

            traj_rc=np.zeros((self.data[ind].shape[0],))
            for i in range(traj_t_span):
                p = self.data[ind][i,:]
                dist = linalg.norm(self.spline - p[None,:], axis = 1)
                pts_error_arr[np.argmin(dist)]+=np.amin(dist)
                pts_sum[np.argmin(dist)]+=self.data[ind][i,:]
                density[np.argmin(dist)]+=1
                traj_rc[i]=np.argmin(dist)
            data_rc.append(traj_rc)
        
        return traj_error_arr,traj_sum,pts_error_arr,pts_sum,density,data_rc

## Trajectory simulation functions 

In [1]:
def cos_corr(xi, Xj, vi):
    D = Xj - xi
    dist = np.linalg.norm(D, axis=1)
    dist[dist == 0] = 1
    D /= dist[:, None]

    v_norm = np.linalg.norm(vi)
    if v_norm == 0:
        v_norm = 1
    
    return D @ vi / v_norm

# X = X or X_spliced/M_s
# V = velocity_S
# nbrs = uns.neighbors (indices)
def corr_kernel(X, V, nbrs, sigma=10, corr_func=cos_corr):
    P = np.zeros((X.shape[0], X.shape[0]))
    for i, x in enumerate(X):
        v, idx = V[i], nbrs[i]

        c = corr_func(x, X[idx], v)
        p = np.exp(c/sigma)
        p /= np.sum(p)
        P[i][idx] = p
    return P

In [2]:
def markov_simulation(T, num_trajs, starts, targets):
    final_trajs = []
    for i in range(num_trajs):
        states = [random.choice(starts)]
        t=0
        while states[t] not in targets:
            p = T[states[t],:]
            temp = np.where(np.random.multinomial(1, p))[0][0]
            if temp in start_pos and len(states)!=1:
                states = [temp]
                t=0
            else:   
                states.append(temp)
                t+=1
        final_trajs.append(states)
    return final_trajs

In [4]:
def markov_simulation2(T, num_trajs, starts, targets):
    final_trajs = []
    for i in range(num_trajs):
        states = [random.choice(starts)]
        t=0
        while states[t] not in targets:
            p = T[states[t],:]
            temp = np.where(np.random.multinomial(1, p))[0][0]
            #if temp in starts and len(states)!=1:
            if temp in starts:
                states = [temp]
                t=0
            else:   
                states.append(temp)
                t+=1
        final_trajs.append(states)
    return final_trajs

In [3]:
# Returns the distance matrix between LAP trajectories
def distanceMatrix(paths):
    toReturn = np.zeros((len(paths), len(paths)))
    row = 0 
    col = 0
    while row < toReturn.shape[0]:
        while col < toReturn.shape[0]:
            dist, _ = fastdtw(paths[row], paths[col])
            toReturn[row, col] = dist
            toReturn[col, row] = dist
            col += 1
        col = row+1
        row += 1
    return toReturn

In [4]:
# Returns the distance matrix between LAP trajectories
def distanceMatrix2(paths, file_name):
    #toReturn = np.zeros((len(paths), len(paths)))
    #toReturn = []
    file = open(file_name, 'w')
    row = 0 
    col = 0
    while row < len(paths):
        toAdd = []
        while col < len(paths):
            dist, _ = fastdtw(paths[row], paths[col])
            toAdd.append(dist)
            #file.write(str(dist) + "\t")
            #toReturn[row, col] = dist
            #toReturn[col, row] = dist
            col += 1
        file.write("\n")
        col = row+1
        row += 1
    return toReturn

In [5]:
def getPath(predecessors, start_index, end_index):
    path = []
    while predecessors[start_index]!=-9999:
        path.append(start_index)
        start_index = predecessors[start_index]
    if predecessors[start_index]!=9999:   
        path.append(end_index)
    return path

In [6]:
def getDistance(coord1, coord2):
    return np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)